In [ ]:
# ==============================================================================
# 🔧 PYTORCH KURULUMU - RTX 5080 (sm_120) DESTEĞİ İÇİN
# ==============================================================================
# RTX 5080 için CUDA 12.8+ gereklidir (sm_120 desteği)
# PyTorch'un en son sürümü CUDA 12.8 ile kurulmalıdır

import sys
import subprocess

def install_pytorch():
    """PyTorch'u RTX 5080 için CUDA 12.8 ile kurar"""
    print("=" * 60)
    print("🔧 PYTORCH KURULUMU - RTX 5080 İÇİN")
    print("=" * 60)
    print("📦 PyTorch + CUDA 12.8 kuruluyor (sm_120 desteği)...")
    print("=" * 60)
    
    # Önce mevcut sürümleri kaldır
    try:
        subprocess.check_call([
            sys.executable, "-m", "pip", "uninstall", "-y", 
            "torch", "torchvision", "torchaudio"
        ])
    except:
        pass
    
    # CUDA 12.8 ile en son PyTorch sürümünü kur
    subprocess.check_call([
        sys.executable, "-m", "pip", "install",
        "torch", "torchvision", "torchaudio",
        "--index-url", "https://download.pytorch.org/whl/cu128"
    ])
    
    print("=" * 60)
    print("✅ Kurulum tamamlandı!")
    print("🔄 Lütfen kernel'ı restart edin (Kernel > Restart)")
    print("=" * 60)

# Kurulumu çalıştır
print("🔧 PyTorch kurulumu başlatılıyor...")
install_pytorch()

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import gc
import warnings
import subprocess
import shutil
from pathlib import Path

# ==============================================================================
# 🔧 OPENMP ÇAKIŞMA ÇÖZÜMÜ (Önce bu ayarlanmalı!)
# ==============================================================================
# OpenMP runtime çakışmasını önlemek için
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
# NumPy threading ayarları
os.environ['OMP_NUM_THREADS'] = '1'

# ==============================================================================
# ⚠️ PYTORCH IMPORT KONTROLÜ - GELİŞMİŞ DLL HATASI ÇÖZÜMÜ
# ==============================================================================
TORCH_AVAILABLE = False

def diagnose_system():
    """Sistem durumunu teşhis eder"""
    print("=" * 70)
    print("🔍 SİSTEM TEŞHİSİ")
    print("=" * 70)
    
    # Python bilgileri
    print(f"\n📌 Python: {sys.version}")
    print(f"📌 Python Executable: {sys.executable}")
    print(f"📌 Python Path: {sys.path[0]}")
    
    # PyTorch kurulum kontrolü
    try:
        result = subprocess.run(
            [sys.executable, "-m", "pip", "show", "torch"],
            capture_output=True, text=True, timeout=10
        )
        if result.returncode == 0:
            lines = result.stdout.split('\n')
            for line in lines:
                if 'Version:' in line:
                    print(f"📦 PyTorch Kurulu: {line.split(':')[1].strip()}")
                if 'Location:' in line:
                    print(f"📁 PyTorch Konumu: {line.split(':')[1].strip()}")
        else:
            print("❌ PyTorch kurulu değil veya erişilemiyor")
    except:
        print("⚠️  PyTorch bilgisi alınamadı")
    
    # Visual C++ Redistributables kontrolü (Windows)
    if sys.platform == 'win32':
        try:
            import winreg
            vc_redist_found = False
            try:
                key = winreg.OpenKey(
                    winreg.HKEY_LOCAL_MACHINE,
                    r"SOFTWARE\Microsoft\Windows\CurrentVersion\Uninstall"
                )
                for i in range(winreg.QueryInfoKey(key)[0]):
                    try:
                        subkey_name = winreg.EnumKey(key, i)
                        subkey = winreg.OpenKey(key, subkey_name)
                        try:
                            display_name = winreg.QueryValueEx(subkey, "DisplayName")[0]
                            if "Visual C++" in display_name and "Redistributable" in display_name:
                                print(f"✅ {display_name}")
                                vc_redist_found = True
                        except:
                            pass
                        subkey.Close()
                    except:
                        pass
                key.Close()
            except:
                pass
            
            if not vc_redist_found:
                print("❌ Visual C++ Redistributables BULUNAMADI!")
                print("   🔗 İndir: https://aka.ms/vs/17/release/vc_redist.x64.exe")
        except:
            print("⚠️  Visual C++ kontrolü yapılamadı")
    
    print("=" * 70)

def deep_clean_pytorch():
    """PyTorch'u tamamen temizler"""
    print("\n🧹 PyTorch derin temizlik başlatılıyor...")
    
    # 1. Pip ile kaldır
    try:
        print("1️⃣  Pip ile kaldırılıyor...")
        subprocess.run(
            [sys.executable, "-m", "pip", "uninstall", "-y", "torch", "torchvision", "torchaudio"],
            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, timeout=60
        )
    except:
        pass
    
    # 2. Cache temizle
    try:
        print("2️⃣  Pip cache temizleniyor...")
        subprocess.run(
            [sys.executable, "-m", "pip", "cache", "purge"],
            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, timeout=30
        )
    except:
        pass
    
    # 3. Python cache temizle
    try:
        print("3️⃣  Python cache temizleniyor...")
        site_packages = None
        for path in sys.path:
            if 'site-packages' in path:
                site_packages = Path(path)
                break
        
        if site_packages:
            torch_dirs = ['torch', 'torchvision', 'torchaudio']
            for dir_name in torch_dirs:
                dir_path = site_packages / dir_name
                if dir_path.exists():
                    try:
                        shutil.rmtree(dir_path)
                        print(f"   ✅ {dir_name} klasörü silindi")
                    except:
                        pass
    except:
        pass
    
    # 4. __pycache__ temizle
    try:
        print("4️⃣  __pycache__ temizleniyor...")
        for path in sys.path:
            cache_path = Path(path) / '__pycache__'
            if cache_path.exists():
                try:
                    shutil.rmtree(cache_path)
                except:
                    pass
    except:
        pass
    
    print("✅ Temizlik tamamlandı!")

def reinstall_pytorch():
    """PyTorch'u RTX 5080 için CUDA 12.8 ile yeniden kurar"""
    print("\n📦 PyTorch yeniden kuruluyor (RTX 5080 için CUDA 12.8)...")
    print("   Bu işlem 5-10 dakika sürebilir...")
    
    try:
        # Conda varsa conda ile dene
        conda_exe = shutil.which('conda')
        if conda_exe:
            print("   Conda ile kurulum deneniyor...")
            try:
                subprocess.run([
                    conda_exe, "install", "-y", 
                    "pytorch", "torchvision", "torchaudio", 
                    "pytorch-cuda=12.8", 
                    "-c", "pytorch", "-c", "nvidia"
                ], timeout=600, check=True)
                print("✅ Conda ile kurulum başarılı!")
                return True
            except:
                print("   Conda kurulumu başarısız, pip deneniyor...")
        
        # Pip ile kur (CUDA 12.8 - RTX 5080 için gerekli)
        print("   Pip ile kurulum deneniyor (CUDA 12.8)...")
        subprocess.run([
            sys.executable, "-m", "pip", "install",
            "torch", "torchvision", "torchaudio",
            "--index-url", "https://download.pytorch.org/whl/cu128",
            "--no-cache-dir"
        ], timeout=600, check=True)
        print("✅ Pip ile kurulum başarılı!")
        return True
    except subprocess.TimeoutExpired:
        print("❌ Kurulum zaman aşımına uğradı")
        return False
    except subprocess.CalledProcessError as e:
        print(f"❌ Kurulum hatası: {e}")
        return False
    except Exception as e:
        print(f"❌ Beklenmeyen hata: {e}")
        return False

# ==============================================================================
# ANA IMPORT DENEMESİ
# ==============================================================================

# RTX 5080 (sm_120) uyumluluk uyarısını filtrele
warnings.filterwarnings('ignore', message='.*CUDA capability sm_120.*', category=UserWarning)

# Önce sistem teşhisi yap
diagnose_system()

try:
    # Circular import hatasını önlemek için cache temizle
    if 'torch' in sys.modules:
        del sys.modules['torch']
        import importlib
        importlib.invalidate_caches()
    
    import torch
    TORCH_AVAILABLE = True
    
    # RTX 5080 uyumluluk kontrolü
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        cuda_capability = torch.cuda.get_device_capability(0)
        cuda_major, cuda_minor = cuda_capability
        
        # sm_120 kontrolü (12.0 = sm_120)
        if cuda_major == 12 and cuda_minor == 0:
            print("\n" + "=" * 70)
            print("⚠️  RTX 5080 (sm_120) UYARISI")
            print("=" * 70)
            print("📌 GPU: RTX 5080 (CUDA Capability: 12.0)")
            print("📌 PyTorch CUDA Versiyonu:", torch.version.cuda)
            if torch.version.cuda and float(torch.version.cuda.split('.')[0] + '.' + torch.version.cuda.split('.')[1]) < 12.8:
                print("❌ PyTorch CUDA 12.8+ gereklidir!")
                print("🔧 Çözüm: CUDA 12.8 ile PyTorch kurun:")
                print("   pip uninstall -y torch torchvision torchaudio")
                print("   pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128")
            else:
                print("✅ CUDA 12.8+ kurulu - RTX 5080 destekleniyor!")
            print("=" * 70)
    
    print("\n" + "=" * 70)
    print("✅ PYTORCH BAŞARIYLA YÜKLENDİ!")
    print("=" * 70)
    print(f"📦 Versiyon: {torch.__version__}")
    print(f"🔧 OpenMP Çakışma: Çözüldü (KMP_DUPLICATE_LIB_OK=TRUE)")
    if torch.cuda.is_available():
        print(f"🎮 CUDA: Kullanılabilir")
        print(f"   CUDA Versiyon: {torch.version.cuda}")
        print(f"   GPU: {torch.cuda.get_device_name(0)}")
        cap = torch.cuda.get_device_capability(0)
        print(f"   CUDA Capability: {cap[0]}.{cap[1]} (sm_{cap[0]}{cap[1]})")
    else:
        print("⚠️  CUDA: Kullanılamıyor (CPU modu)")
    print("=" * 70)
    
except (OSError, AttributeError, ImportError) as e:
    error_msg = str(e)
    print("\n" + "=" * 70)
    print("❌ PYTORCH YÜKLENEMEDİ!")
    print("=" * 70)
    print(f"📋 Hata: {error_msg}")
    
    # DLL hatası kontrolü
    if "fbgemm.dll" in error_msg or "dll" in error_msg.lower() or "WinError 126" in error_msg:
        print("\n🔍 SORUN: DLL bağımlılık hatası")
        print("\n" + "=" * 70)
        print("🔧 ÇÖZÜM ADIMLARI (Sırayla Deneyin)")
        print("=" * 70)
        
        print("\n1️⃣  VISUAL C++ REDISTRIBUTABLES KURUN (ZORUNLU!)")
        print("   🔗 İndir: https://aka.ms/vs/17/release/vc_redist.x64.exe")
        print("   ⚠️  Kurulumdan sonra bilgisayarı yeniden başlatın!")
        
        print("\n2️⃣  DERİN TEMİZLİK VE YENİDEN KURULUM")
        print("   Aşağıdaki kodu çalıştırın:")
        print("   " + "-" * 66)
        print("   deep_clean_pytorch()")
        print("   reinstall_pytorch()")
        print("   " + "-" * 66)
        print("   Sonra kernel'ı restart edin ve tekrar deneyin.")
        
        print("\n3️⃣  MANUEL TERMINAL KOMUTLARI (RTX 5080 için CUDA 12.8)")
        print("   Terminal'de (conda activate seg5080 sonrası):")
        print("   " + "-" * 66)
        print("   pip uninstall -y torch torchvision torchaudio")
        print("   pip cache purge")
        print("   pip install torch torchvision torchaudio \\")
        print("       --index-url https://download.pytorch.org/whl/cu128 --no-cache-dir")
        print("   " + "-" * 66)
        print("   ⚠️  ÖNEMLİ: RTX 5080 için CUDA 12.8 gereklidir (sm_120 desteği)")
        
        print("\n4️⃣  ALTERNATİF: CONDA İLE KUR (Bağımlılıkları otomatik halleder)")
        print("   Terminal'de çalıştırın:")
        print("   " + "-" * 66)
        print("   conda activate seg5080")
        print("   conda install pytorch torchvision torchaudio pytorch-cuda=12.8 \\")
        print("       -c pytorch -c nvidia")
        print("   " + "-" * 66)
        print("   ⚠️  ÖNEMLİ: RTX 5080 için pytorch-cuda=12.8 gereklidir")
        
        print("\n5️⃣  GEÇİCİ ÇÖZÜM: CPU-ONLY VERSİYON (Test için)")
        print("   Eğer Visual C++ kuramıyorsanız, CPU-only ile test edebilirsiniz:")
        print("   " + "-" * 66)
        print("   pip uninstall -y torch torchvision torchaudio")
        print("   pip install torch torchvision torchaudio")
        print("   " + "-" * 66)
        print("   ⚠️  Not: Bu versiyon GPU kullanmaz, sadece test içindir!")
        
        print("\n" + "=" * 70)
        print("⚠️  ÖNEMLİ: Visual C++ Redistributables olmadan çalışmaz!")
        print("=" * 70)
        print("\n📝 ŞU ANDA YAPMANIZ GEREKENLER:")
        print("   1. Visual C++ Redistributables'ı kurun (yukarıdaki link)")
        print("   2. Bilgisayarı yeniden başlatın")
        print("   3. Bu cell'i tekrar çalıştırın")
        print("=" * 70)
    
    TORCH_AVAILABLE = False
    # Hata mesajını göster ama execution'ı durdurma
    raise

# import torch_directml  <-- BUNU SİL, ARTIK İHTİYACIN YOK

# ==============================================================================
# 🚀 RTX 5080 OPTİMİZASYON AYARLARI (Arkadaşının dosyasından uyarlandı)
# ==============================================================================

if TORCH_AVAILABLE:
    # 1. Ortam Değişkenleri (DirectML ve Kısıtlamaları Kaldırıyoruz)
    os.environ["CUDA_LAUNCH_BLOCKING"] = "0" 
    # CUDNN'i tekrar açıyoruz (Senin dosyada kapalıydı)
    if "TORCH_CUDNN_DISABLED" in os.environ:
        del os.environ["TORCH_CUDNN_DISABLED"]

    # 2. GPU Hızlandırma Ayarları
    torch.backends.cudnn.benchmark = True       # En hızlı algoritmayı seçer
    torch.backends.cudnn.deterministic = False  # Hız için deterministik modu kapa
    torch.backends.cuda.matmul.allow_tf32 = True # RTX 50 serisi için TF32 aç (Büyük Hız Artışı!)
    torch.backends.cudnn.allow_tf32 = True

    # 3. Device Tanımlama (DirectML yerine Native CUDA)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    print("=" * 60)
    print(f"🎮 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU ❌'}")
    print(f"🔧 CUDA Sürümü: {torch.version.cuda}")
    print(f"🚀 Device: {device}")
    print("⚡ TF32 Hızlandırma: Aktif")
    print("=" * 60)
    
    # RAM temizliği
    gc.collect()
    torch.cuda.empty_cache()
else:
    print("=" * 60)
    print("⚠️  PyTorch yüklenemedi - Yukarıdaki hata mesajına bakın")
    print("=" * 60)

In [ ]:
# =============================================================================
# HUCRE 1.5: LOSS FUNCTIONS (DiceLoss, FocalLoss, DiceFocalLoss)
# =============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F

class DiceLoss(nn.Module):
    """
    Dice Loss for segmentation - directly optimizes IoU metric
    Formula: 1 - (2 * intersection + smooth) / (pred + target + smooth)
    """
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, inputs, targets):
        """
        Args:
            inputs: [B, C, H, W] logits
            targets: [B, H, W] class indices
        """
        num_classes = inputs.shape[1]
        
        # Softmax to get probabilities
        inputs_soft = torch.softmax(inputs, dim=1)
        
        # One-hot encode targets
        targets_one_hot = F.one_hot(targets.long(), num_classes=num_classes)
        targets_one_hot = targets_one_hot.permute(0, 3, 1, 2).float()
        
        # Calculate Dice loss per class
        dice_loss = 0.0
        for c in range(num_classes):
            pred_c = inputs_soft[:, c].contiguous().view(-1)
            target_c = targets_one_hot[:, c].contiguous().view(-1)
            
            intersection = (pred_c * target_c).sum()
            dice_score = (2. * intersection + self.smooth) / (
                pred_c.sum() + target_c.sum() + self.smooth
            )
            dice_loss += (1 - dice_score)
        
        return dice_loss / num_classes


class FocalLoss(nn.Module):
    """
    Focal Loss for addressing class imbalance in segmentation
    Formula: FL(p_t) = -alpha_t * (1 - p_t)^gamma * log(p_t)
    
    Reference: https://arxiv.org/abs/1708.02002
    """
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        """
        Args:
            alpha: Class weights tensor [weight_class0, weight_class1, ...]
            gamma: Focusing parameter (higher = more focus on hard examples)
            reduction: 'mean', 'sum', or 'none'
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        """
        Args:
            inputs: [B, C, H, W] logits
            targets: [B, H, W] class indices
        """
        # Flatten spatial dimensions
        B, C, H, W = inputs.shape
        inputs_flat = inputs.permute(0, 2, 3, 1).contiguous().view(-1, C)
        targets_flat = targets.view(-1).long()
        
        # Cross entropy (without reduction)
        ce_loss = F.cross_entropy(inputs_flat, targets_flat, reduction='none')
        
        # Get probability of correct class
        p = torch.softmax(inputs_flat, dim=1)
        p_t = p.gather(1, targets_flat.unsqueeze(1)).squeeze(1)
        
        # Focal weight: (1 - p_t)^gamma
        focal_weight = (1 - p_t) ** self.gamma
        
        # Apply focal weighting
        focal_loss = focal_weight * ce_loss
        
        # Apply class weights if provided
        if self.alpha is not None:
            alpha_t = self.alpha.to(inputs.device).gather(0, targets_flat)
            focal_loss = alpha_t * focal_loss
        
        # Reduction
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


class DiceFocalLoss(nn.Module):
    """
    Combined Dice + Focal Loss for segmentation
    Best for: Binary segmentation with class imbalance
    
    Dice: Optimizes IoU directly (good for region overlap)
    Focal: Handles class imbalance (focuses on hard examples)
    """
    def __init__(self, dice_weight=0.6, focal_weight=0.4,
                 focal_alpha=None, focal_gamma=2.0, smooth=1.0):
        """
        Args:
            dice_weight: Weight for Dice loss (0-1)
            focal_weight: Weight for Focal loss (0-1)
            focal_alpha: Class weights for Focal loss
            focal_gamma: Focusing parameter for Focal loss
            smooth: Smoothing factor for Dice loss
        """
        super(DiceFocalLoss, self).__init__()
        self.dice_weight = dice_weight
        self.focal_weight = focal_weight
        self.dice_loss = DiceLoss(smooth=smooth)
        self.focal_loss = FocalLoss(alpha=focal_alpha, gamma=focal_gamma)
    
    def forward(self, inputs, targets):
        dice = self.dice_loss(inputs, targets)
        focal = self.focal_loss(inputs, targets)
        combined = self.dice_weight * dice + self.focal_weight * focal
        return combined


print("Loss Functions tanimlandı:")
print("  - DiceLoss: IoU optimizasyonu")
print("  - FocalLoss: Class imbalance icin")
print("  - DiceFocalLoss: Kombinasyon (Onerilen)")

In [ ]:
# =============================================================================
# HUCRE 2: AYARLAR (CONFIG) - GENISLETILMIS
# =============================================================================
from utils import create_training_folder

# Notebook adı (versiyonlama için)
NOTEBOOK_NAME = "segformer_5080_final"

# Versiyonlu klasör yapısı oluştur
base_dir, models_dir, data_dir, viz_dir, version = create_training_folder(NOTEBOOK_NAME)

CONFIG = {
    # =============================================
    # 1. VERI AYARLARI
    # =============================================
    'root_dir': r"C:\AI_DATA\SEMI_TRUTHS_extracted", 
    #'csv_path': "segmentation_dataset_balanced.csv",  # Tek CSV dosyası (train/val/test içinde)
    # VEYA ayrı CSV dosyaları kullanılacaksa:
    'train_csv_path': "dataset_splits/fake_only_split/fake_train.csv",
    'val_csv_path': "dataset_splits/fake_only_split/fake_val.csv",
    'test_csv_path': "dataset_splits/fake_only_split/fake_test.csv",
    'dataset_name': "SEMI_TRUTHS",  # Dataset adı (tracking için)
    
    # Data Ratio: Önceden belirlenmiş train/val/test CSV'lerinden ne kadarının kullanılacağı
    'train_data_ratio': 0.01,  # Train CSV'den kullanılacak oran (0.0-1.0, 1.0 = tümü)
    'val_data_ratio': 0.01,    # Val CSV'den kullanılacak oran (0.0-1.0, 1.0 = tümü)
    'test_data_ratio': 0.01,   # Test CSV'den kullanılacak oran (0.0-1.0, 1.0 = tümü, opsiyonel)
    
    # =============================================
    # 2. MODEL AYARLARI
    # =============================================
    'model_name': "nvidia/mit-b4", 
    'model_variant': "mit-b4",  # Model variant (b0, b1, b2, b3, b4, b5)
    'output_dir': models_dir,  # Versiyonlu klasöre kaydet
    'img_size': 512,
    'num_classes': 2,  # 0: Background, 1: Fake
    
    # =============================================
    # 3. EGITIM HIPERPARAMETRELERI
    # =============================================
    'batch_size': 16,
    'gradient_accumulation': 2,  # Effective batch = 16
    'num_epochs': 10,
    'learning_rate': 6e-5,
    'weight_decay': 0.01,
    'seed': 42,
    
    # =============================================
    # 4. LOSS FUNCTION AYARLARI (YENİ!)
    # =============================================
    # Secenekler: 'crossentropy', 'focal', 'dice', 'dice_focal'
    'loss_type': 'dice_focal',
    
    # Dice Loss ayarları
    'dice_weight': 0.6,        # Dice loss ağırlığı (0-1)
    'dice_smooth': 1.0,        # Smoothing factor
    
    # Focal Loss ayarları
    'focal_weight': 0.4,       # Focal loss ağırlığı (0-1)
    'focal_gamma': 2.0,        # Focusing parameter (2.0 standart)
    'focal_alpha': [0.25, 0.75],  # Class weights [Background, Fake]
    
    # =============================================
    # 5. CIKTI KLASORLERI (Versiyonlu)
    # =============================================
    'base_dir': base_dir,
    'models_dir': models_dir,
    'data_dir': data_dir,
    'viz_dir': viz_dir,
    'version': version,
}

# Seed sabitleme
torch.manual_seed(CONFIG['seed'])
np.random.seed(CONFIG['seed'])

print(f"\n{'='*70}")
print(f"EGITIM OTURUMU: {NOTEBOOK_NAME} - Version {version}")
print(f"{'='*70}")
print(f"  Model:     {CONFIG['model_name']}")
print(f"  Loss:      {CONFIG['loss_type']}")
if CONFIG['loss_type'] == 'dice_focal':
    print(f"             Dice weight: {CONFIG['dice_weight']}, Focal weight: {CONFIG['focal_weight']}")
    print(f"             Focal gamma: {CONFIG['focal_gamma']}, Alpha: {CONFIG['focal_alpha']}")
print(f"  Batch:     {CONFIG['batch_size']} x {CONFIG['gradient_accumulation']} = {CONFIG['batch_size'] * CONFIG['gradient_accumulation']}")
print(f"  Epochs:    {CONFIG['num_epochs']}")
print(f"  LR:        {CONFIG['learning_rate']}")
print(f"{'='*70}")
print(f"  Models:    {models_dir}")
print(f"  Data:      {data_dir}")
print(f"  Viz:       {viz_dir}")
print(f"{'='*70}")

In [ ]:
#HÜCRE 3: Dataset Sınıfı (TAMAMEN DÜZELTİLMİŞ)
import os
import pandas as pd
import albumentations as A
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

class FakeImageSegmentationDataset(Dataset):
    def __init__(self, df, processor, is_train=True):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.is_train = is_train
        
        # Gerekli kolonları kontrol et
        required_cols = ['image_path', 'mask_path']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(
                f"❌ Dataset'te gerekli kolonlar eksik: {missing_cols}\n"
                f"   Mevcut kolonlar: {list(df.columns)}\n"
                f"   💡 Önce mask_path kolonunu oluşturun (HÜCRE 4.5'teki kod çalıştırılmalı)"
            )
        
        # Metadata kolonlarını belirle (image_path ve mask_path hariç)
        metadata_cols = [col for col in df.columns 
                         if col not in ['image_path', 'mask_path']]
        self.metadata_cols = metadata_cols
        
        # Augmentation (Veri çeşitlendirme - Sadece eğitimde)
        if is_train:
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.RandomRotate90(p=0.5),
                A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
                A.OneOf([
                    A.GridDistortion(p=0.3),
                    A.ElasticTransform(p=0.3),
                ], p=0.3),
                A.RandomBrightnessContrast(p=0.3),
            ], is_check_shapes=False)
        else:
            self.transform = None
    
    def __len__(self):
        return len(self.df)
    
    def get_metadata(self, idx):
        """Get metadata for a given index"""
        row = self.df.iloc[idx]
        metadata = {}
        for col in self.metadata_cols:
            value = row.get(col, None)
            # Convert numpy types to Python native types for JSON serialization
            if isinstance(value, (np.integer, np.floating)):
                metadata[col] = value.item()
            elif isinstance(value, np.ndarray):
                metadata[col] = value.tolist()
            else:
                metadata[col] = value
        return metadata
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        try:
            # Görüntü (RGB)
            image_path = row.get('image_path')
            if pd.isna(image_path) or not os.path.exists(str(image_path)):
                raise FileNotFoundError(f"Image path bulunamadı veya geçersiz: {image_path}")
            image = Image.open(image_path).convert("RGB")
            
            # Maske (L - Grayscale)
            mask_path = row.get('mask_path')
            if pd.isna(mask_path) or mask_path is None:
                raise ValueError(f"mask_path eksik veya None (Satır {idx})")
            if not os.path.exists(str(mask_path)):
                raise FileNotFoundError(f"Mask dosyası bulunamadı: {mask_path}")
            mask = Image.open(mask_path).convert("L")
            
            # Boyut Kontrolü
            if image.size != mask.size:
                mask = mask.resize(image.size, Image.NEAREST)
            
            image_np = np.array(image)
            mask_np = np.array(mask)
            
            # 🔥 DÜZELTİLMİŞ MASKE MANTIĞI 🔥
            # Simsiyah (0) olmayan her şeyi "FAKE (1)" yap.
            mask_np = np.where(mask_np > 0, 1, 0).astype(np.uint8)
            
            # Augmentation
            if self.transform and self.is_train:
                augmented = self.transform(image=image_np, mask=mask_np)
                image_np = augmented['image']
                mask_np = augmented['mask']
            
            # Processor
            encoded = self.processor(
                images=image_np, 
                segmentation_maps=mask_np, 
                return_tensors="pt"
            )
            
            # Batch boyutunu sıkıştır
            for k, v in encoded.items():
                encoded[k] = v.squeeze(0)
            
            # Metadata ekle (evaluation için)
            encoded['metadata'] = self.get_metadata(idx)
            encoded['idx'] = idx
                
            return encoded
            
        except Exception as e:
            print(f"⚠️ Hata (Satır {idx}): {e}")
            # Hata durumunda bir sonraki veriyi dene
            return self.__getitem__((idx + 1) % len(self.df))

print("✅ Dataset sınıfı tanımlandı (Tüm kütüphanelerle birlikte).")

In [ ]:
#HÜCRE 4: Veri Yükleme ve Hazırlık
# Önceden belirlenmiş train/val/test CSV'lerinden veri yükleme

# Ayrı CSV dosyaları var mı kontrol et
if 'train_csv_path' in CONFIG and 'val_csv_path' in CONFIG:
    # Ayrı CSV dosyaları kullanılıyor
    print("="*60)
    print("AYRI CSV DOSYALARINDAN VERİ YÜKLEME")
    print("="*60)
    
    # Train CSV
    if not os.path.exists(CONFIG['train_csv_path']):
        raise FileNotFoundError(f"❌ Train CSV bulunamadı: {CONFIG['train_csv_path']}")
    train_df_full = pd.read_csv(CONFIG['train_csv_path'])
    print(f"📄 Train CSV: {len(train_df_full)} görsel")
    
    # Val CSV
    if not os.path.exists(CONFIG['val_csv_path']):
        raise FileNotFoundError(f"❌ Val CSV bulunamadı: {CONFIG['val_csv_path']}")
    val_df_full = pd.read_csv(CONFIG['val_csv_path'])
    print(f"📄 Val CSV: {len(val_df_full)} görsel")
    
    # Test CSV (opsiyonel)
    test_df_full = None
    if 'test_csv_path' in CONFIG and os.path.exists(CONFIG['test_csv_path']):
        test_df_full = pd.read_csv(CONFIG['test_csv_path'])
        print(f"📄 Test CSV: {len(test_df_full)} görsel")
    
    # Data Ratio uygula
    train_data_ratio = CONFIG.get('train_data_ratio', 1.0)
    val_data_ratio = CONFIG.get('val_data_ratio', 1.0)
    test_data_ratio = CONFIG.get('test_data_ratio', 1.0)
    
    if train_data_ratio < 1.0:
        original_train = len(train_df_full)
        train_df = train_df_full.sample(frac=train_data_ratio, random_state=CONFIG['seed']).reset_index(drop=True)
        print(f"📊 Train Data Ratio: {train_data_ratio*100:.1f}% ({len(train_df)}/{original_train} görsel)")
    else:
        train_df = train_df_full
        print(f"📊 Train Data Ratio: {train_data_ratio*100:.1f}% (Tümü kullanılıyor)")
    
    if val_data_ratio < 1.0:
        original_val = len(val_df_full)
        val_df = val_df_full.sample(frac=val_data_ratio, random_state=CONFIG['seed']).reset_index(drop=True)
        print(f"📊 Val Data Ratio: {val_data_ratio*100:.1f}% ({len(val_df)}/{original_val} görsel)")
    else:
        val_df = val_df_full
        print(f"📊 Val Data Ratio: {val_data_ratio*100:.1f}% (Tümü kullanılıyor)")
    
    if test_df_full is not None and test_data_ratio < 1.0:
        original_test = len(test_df_full)
        test_df = test_df_full.sample(frac=test_data_ratio, random_state=CONFIG['seed']).reset_index(drop=True)
        print(f"📊 Test Data Ratio: {test_data_ratio*100:.1f}% ({len(test_df)}/{original_test} görsel)")
    elif test_df_full is not None:
        test_df = test_df_full
        print(f"📊 Test Data Ratio: {test_data_ratio*100:.1f}% (Tümü kullanılıyor)")
    
else:
    # Tek CSV dosyası kullanılıyor (eski yöntem - train/val split yapılacak)
    print("="*60)
    print("TEK CSV DOSYASINDAN VERİ YÜKLEME (Train/Val Split)")
    print("="*60)
    print("⚠️  UYARI: Ayrı train/val/test CSV'leri kullanmak için CONFIG'e train_csv_path ve val_csv_path ekleyin!")
    
    if not os.path.exists(CONFIG['csv_path']):
        raise FileNotFoundError(f"❌ CSV bulunamadı: {CONFIG['csv_path']}")
    
    df = pd.read_csv(CONFIG['csv_path'])
    print(f"📄 Toplam Veri (CSV'den): {len(df)} görsel")
    
    # Data Ratio uygula (train ve val için aynı oran)
    train_data_ratio = CONFIG.get('train_data_ratio', 1.0)
    val_data_ratio = CONFIG.get('val_data_ratio', 1.0)
    
    # Önce data ratio uygula
    if train_data_ratio < 1.0 or val_data_ratio < 1.0:
        # Ortalama ratio kullan (veya train_data_ratio'yu kullan)
        data_ratio = train_data_ratio if train_data_ratio < 1.0 else val_data_ratio
        original_size = len(df)
        df = df.sample(frac=data_ratio, random_state=CONFIG['seed']).reset_index(drop=True)
        print(f"📊 Data Ratio Uygulandı: {data_ratio*100:.1f}% ({len(df)}/{original_size} görsel)")
    
    # Train/Val Split (varsayılan %90/%10)
    df_shuffled = df.sample(frac=1, random_state=CONFIG['seed']).reset_index(drop=True)
    train_size = int(0.9 * len(df_shuffled))  # Varsayılan %90 train
    
    train_df = df_shuffled[:train_size]
    val_df = df_shuffled[train_size:]
    test_df = None

# df'yi oluştur (tüm dataframes'i birleştir) - Cell 5 için gerekli
# Eğer df zaten varsa (tek CSV durumu), değiştirme
try:
    _ = df  # df var mı kontrol et
except NameError:
    # df yok, ayrı CSV'lerden yüklendi, birleştir
    dfs_to_combine = [train_df, val_df]
    if test_df is not None:
        dfs_to_combine.append(test_df)
    df = pd.concat(dfs_to_combine, ignore_index=True)
    print(f"📋 Birleştirilmiş DataFrame oluşturuldu: {len(df)} satır")

print("\n" + "="*60)
print("VERİ SETİ ÖZETİ")
print("="*60)
print(f"📊 Eğitim Seti:    {len(train_df)} görsel")
print(f"🧪 Doğrulama Seti: {len(val_df)} görsel")
if test_df is not None:
    print(f"🧪 Test Seti:      {len(test_df)} görsel")
print(f"📈 Toplam Kullanılan: {len(train_df) + len(val_df) + (len(test_df) if test_df is not None else 0)} görsel")
print("="*60)

# =============================================================================
# PATH KOLONLARINI OLUŞTUR (image_path ve mask_path)
# =============================================================================
print("\n" + "="*60)
print("PATH KOLONLARINI OLUŞTURMA")
print("="*60)

# Helper fonksiyon: DataFrame'e image_path ekle
def add_image_path_column(dataframe, df_name="df"):
    """DataFrame'e image_path kolonu ekler"""
    if 'image_path' not in dataframe.columns:
        if 'fake_img_path' in dataframe.columns:
            dataframe['image_path'] = dataframe['fake_img_path']
            print(f"✅ {df_name}: image_path oluşturuldu (fake_img_path kullanıldı)")
        elif 'img_path' in dataframe.columns:
            dataframe['image_path'] = dataframe['img_path']
            print(f"✅ {df_name}: image_path oluşturuldu (img_path kullanıldı)")
        else:
            raise ValueError(f"❌ {df_name}: image_path oluşturulamadı! Mevcut kolonlar: {list(dataframe.columns)}")
    else:
        print(f"✅ {df_name}: image_path zaten mevcut")

# 1. image_path kolonunu oluştur (df için)
add_image_path_column(df, "df")

# train_df, val_df ve test_df'ye de image_path ekle
if 'train_df' in locals():
    add_image_path_column(train_df, "train_df")
if 'val_df' in locals():
    add_image_path_column(val_df, "val_df")
if 'test_df' in locals() and test_df is not None:
    add_image_path_column(test_df, "test_df")

# 2. mask_path kolonunu oluştur (mask_id ve parent_dataset'ten)
if 'mask_path' not in df.columns:
    # Mask path oluşturma fonksiyonu
    def get_mask_path(mask_id, parent_dataset, root_dir=None):
        """
        mask_id ve parent_dataset'ten mask path'i oluşturur
        
        Args:
            mask_id: Mask ID (örn: 'instance_000_ADE_frame_00000004')
            parent_dataset: Dataset adı (örn: 'ADE20K', 'CelebAHQ')
            root_dir: Root directory (CONFIG['root_dir'] kullanılır)
        
        Returns:
            str: Mask dosyasının tam yolu veya None
        """
        if pd.isna(mask_id) or pd.isna(parent_dataset):
            return None
        
        if root_dir is None:
            root_dir = CONFIG.get('root_dir', '')
        
        # Mask'ler genellikle {root_dir}/original/masks/{parent_dataset}_masks_{0,1,2}/ altında
        # Önce masks klasörünü bul
        masks_base = os.path.join(root_dir, 'original', 'masks')
        if not os.path.exists(masks_base):
            # Alternatif: root_dir direkt masks içeriyor olabilir
            masks_base = os.path.join(root_dir, 'masks')
            if not os.path.exists(masks_base):
                masks_base = root_dir  # Son çare: root_dir direkt
        
        # Her dataset için 3 klasör var (_0, _1, _2)
        possible_folders = [
            os.path.join(masks_base, f"{parent_dataset}_masks_0"),
            os.path.join(masks_base, f"{parent_dataset}_masks_1"),
            os.path.join(masks_base, f"{parent_dataset}_masks_2")
        ]
        
        # Her klasörde mask'i ara
        for folder in possible_folders:
            mask_path = os.path.join(folder, f"{mask_id}.png")
            if os.path.exists(mask_path):
                return mask_path
        
        # Bulunamazsa None döndür
        return None
    
    # mask_id ve parent_dataset kolonlarını kontrol et
    if 'mask_id' not in df.columns:
        raise ValueError(f"❌ mask_id kolonu bulunamadı! Mevcut kolonlar: {list(df.columns)}")
    
    if 'parent_dataset' not in df.columns:
        # Alternatif: 'dataset' kolonu olabilir
        if 'dataset' in df.columns:
            df['parent_dataset'] = df['dataset']
            print("✅ parent_dataset oluşturuldu: dataset kolonu kullanıldı")
        else:
            raise ValueError(f"❌ parent_dataset veya dataset kolonu bulunamadı! Mevcut kolonlar: {list(df.columns)}")
    
    # mask_path'leri oluştur
    print("🔄 mask_path kolonları oluşturuluyor...")
    root_dir = CONFIG.get('root_dir', '')
    print(f"   Root dir: {root_dir}")
    
    # İlk birkaç örnek için debug bilgisi
    if len(df) > 0:
        sample_row = df.iloc[0]
        sample_mask_id = sample_row.get('mask_id')
        sample_parent_dataset = sample_row.get('parent_dataset')
        print(f"   Örnek mask_id: {sample_mask_id}")
        print(f"   Örnek parent_dataset: {sample_parent_dataset}")
        
        # Mask base path'leri kontrol et
        masks_base_options = [
            os.path.join(root_dir, 'original', 'masks'),
            os.path.join(root_dir, 'masks'),
            root_dir
        ]
        for masks_base in masks_base_options:
            if os.path.exists(masks_base):
                print(f"   ✅ Masks base bulundu: {masks_base}")
                # İlk dataset klasörünü kontrol et
                if sample_parent_dataset:
                    sample_folder = os.path.join(masks_base, f"{sample_parent_dataset}_masks_0")
                    if os.path.exists(sample_folder):
                        print(f"   ✅ Örnek klasör bulundu: {sample_folder}")
                    else:
                        print(f"   ⚠️  Örnek klasör bulunamadı: {sample_folder}")
                break
        else:
            print(f"   ⚠️  Hiçbir masks base bulunamadı!")
    
    mask_paths = []
    missing_count = 0
    
    for idx, row in df.iterrows():
        mask_id = row.get('mask_id')
        parent_dataset = row.get('parent_dataset')
        mask_path = get_mask_path(mask_id, parent_dataset, root_dir)
        
        if mask_path is None:
            missing_count += 1
            mask_paths.append(None)
        else:
            mask_paths.append(mask_path)
    
    df['mask_path'] = mask_paths
    
    if missing_count > 0:
        print(f"⚠️  {missing_count} mask path bulunamadı ({missing_count/len(df)*100:.2f}%)")
        if missing_count == len(df):
            print(f"   ⚠️  TÜM MASK PATH'LER BULUNADI! Root dir veya mask yapısı yanlış olabilir.")
            print(f"   💡 CONFIG['root_dir'] değerini kontrol edin: {root_dir}")
        # Eksik mask'leri temizle
        df = df.dropna(subset=['mask_path']).reset_index(drop=True)
        print(f"🧹 Eksik mask'ler temizlendi: {len(df)} satır kaldı")
    else:
        print(f"✅ Tüm mask path'ler başarıyla oluşturuldu")
    
    # train_df, val_df ve test_df'ye de mask_path ekle
    # Ayrı CSV'lerden yüklendiğinde bunlara da eklememiz gerekiyor
    dataframes_to_update = []
    if 'train_df' in locals():
        dataframes_to_update.append(('train_df', train_df))
    if 'val_df' in locals():
        dataframes_to_update.append(('val_df', val_df))
    if 'test_df' in locals() and test_df is not None:
        dataframes_to_update.append(('test_df', test_df))
    
    # Her DataFrame için mask_path ekle
    for df_name, df_to_update in dataframes_to_update:
        # mask_path kolonu yoksa veya çoğu değer None/boşsa yeniden oluştur
        needs_mask_path = False
        if 'mask_path' not in df_to_update.columns:
            needs_mask_path = True
        else:
            # mask_path kolonu varsa ama çoğu değer None/boşsa yeniden oluştur
            valid_mask_paths = df_to_update['mask_path'].notna() & (df_to_update['mask_path'] != '')
            valid_count = valid_mask_paths.sum()
            if valid_count < len(df_to_update) * 0.5:  # %50'den az geçerli ise
                print(f"   ⚠️  {df_name}: mask_path kolonu var ama çoğu değer eksik ({valid_count}/{len(df_to_update)}), yeniden oluşturuluyor...")
                needs_mask_path = True
        
        if needs_mask_path:
            print(f"🔄 {df_name} için mask_path ekleniyor...")
            
            # mask_id kontrolü
            if 'mask_id' not in df_to_update.columns:
                raise ValueError(f"❌ {df_name}: mask_id kolonu bulunamadı! Mevcut kolonlar: {list(df_to_update.columns)}")
            
            # parent_dataset kontrolü
            if 'parent_dataset' not in df_to_update.columns:
                if 'dataset' in df_to_update.columns:
                    df_to_update['parent_dataset'] = df_to_update['dataset']
                    print(f"   ✅ {df_name}: parent_dataset oluşturuldu (dataset kolonu kullanıldı)")
                else:
                    raise ValueError(f"❌ {df_name}: parent_dataset veya dataset kolonu bulunamadı! Mevcut kolonlar: {list(df_to_update.columns)}")
            
            mask_paths_sub = []
            missing_count_sub = 0
            for idx, row in df_to_update.iterrows():
                mask_id = row.get('mask_id')
                parent_dataset = row.get('parent_dataset')
                
                # mask_id veya parent_dataset eksikse None ekle
                if pd.isna(mask_id) or pd.isna(parent_dataset):
                    missing_count_sub += 1
                    mask_paths_sub.append(None)
                else:
                    mask_path = get_mask_path(mask_id, parent_dataset, root_dir)
                    if mask_path is None:
                        missing_count_sub += 1
                        mask_paths_sub.append(None)
                    else:
                        mask_paths_sub.append(mask_path)
            
            df_to_update['mask_path'] = mask_paths_sub
            
            if missing_count_sub > 0:
                print(f"   ⚠️  {df_name}: {missing_count_sub} mask path bulunamadı ({missing_count_sub/len(df_to_update)*100:.2f}%)")
                # Eksik mask'leri temizle
                df_to_update.dropna(subset=['mask_path'], inplace=True)
                df_to_update.reset_index(drop=True, inplace=True)
                print(f"   🧹 {df_name}: Eksik mask'ler temizlendi, {len(df_to_update)} satır kaldı")
            else:
                print(f"   ✅ {df_name}: Tüm mask path'ler başarıyla oluşturuldu")
        else:
            # mask_path zaten mevcut ve geçerli
            valid_count = (df_to_update['mask_path'].notna() & (df_to_update['mask_path'] != '')).sum()
            print(f"   ✅ {df_name}: mask_path zaten mevcut ({valid_count}/{len(df_to_update)} geçerli)")
    
    # Eğer tek CSV'den split edildiyse, yeniden split yap
    if len(dataframes_to_update) == 0 and 'train_df' in locals() and 'val_df' in locals():
        # df'den mask_path eklenmiş, train/val split'i yeniden yap
        if len(df) > 0:
            train_size = int(0.9 * len(df))  # Varsayılan %90 train
            df_shuffled = df.sample(frac=1, random_state=CONFIG['seed']).reset_index(drop=True)
            train_df = df_shuffled[:train_size]
            val_df = df_shuffled[train_size:]
            print(f"🔄 train_df ve val_df mask_path ile güncellendi")
    
    print("✅ mask_path kolonu oluşturuldu")
else:
    print("✅ mask_path zaten mevcut")

print("="*60)

In [ ]:
# =============================================================================
# HÜCRE 4.5: CSV VALİDASYON KONTROLÜ (Geliştirilmiş)
# =============================================================================
# Bu hücre, CSV'nin doğru yüklendiğini ve kullanıldığını kontrol eder
# HÜCRE 4'ten sonra çalıştırın

print("="*60)
print("CSV VALİDASYON KONTROLÜ")
print("="*60)

# 1. DataFrame'in varlığını kontrol et
if 'df' not in locals() or df is None:
    raise ValueError("❌ DataFrame 'df' bulunamadı! Önce HÜCRE 4'ü çalıştırın.")

# 2. Gerekli kolonları kontrol et
required_cols = ['image_path', 'mask_path']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"❌ EKSİK KOLONLAR: {missing_cols}")
    print(f"   Mevcut kolonlar: {list(df.columns)}")
    raise ValueError(f"CSV'de gerekli kolonlar eksik: {missing_cols}")
else:
    print(f"✅ Gerekli kolonlar mevcut: {required_cols}")

# 3. Boş değer kontrolü
print(f"\n📊 Boş Değer Analizi:")
has_nulls = False
for col in required_cols:
    null_count = df[col].isnull().sum()
    if null_count > 0:
        print(f"  ⚠️  {col}: {null_count} boş değer ({null_count/len(df)*100:.2f}%)")
        has_nulls = True
    else:
        print(f"  ✅ {col}: Boş değer yok")

# 4. Train/Val DataFrame'lerini kontrol et
if 'train_df' not in locals() or 'val_df' not in locals():
    print("\n⚠️  train_df veya val_df bulunamadı!")
else:
    print(f"\n📊 Train/Val Split Kontrolü:")
    print(f"  ✅ Train: {len(train_df)} satır")
    print(f"  ✅ Val:   {len(val_df)} satır")
    print(f"  ✅ Toplam: {len(train_df) + len(val_df)} satır")
    
    # Train/Val'de gerekli kolonlar var mı?
    for col in required_cols:
        if col not in train_df.columns or col not in val_df.columns:
            print(f"  ❌ {col} train_df veya val_df'de eksik!")
        else:
            print(f"  ✅ {col} her iki DataFrame'de mevcut")

# 5. Path geçerliliği örnek kontrolü (ilk 5 satır)
print(f"\n🔍 Path Geçerliliği Örnek Kontrolü (ilk 5 satır):")
if len(df) == 0:
    print("  ⚠️  DataFrame boş! Path kontrolü yapılamıyor.")
else:
    sample_size = min(5, len(df))
    valid_count = 0
    for idx in range(sample_size):
        img_path = df.iloc[idx]['image_path']
        mask_path = df.iloc[idx]['mask_path']
        
        img_exists = os.path.exists(str(img_path)) if pd.notna(img_path) else False
        mask_exists = os.path.exists(str(mask_path)) if pd.notna(mask_path) else False
        
        if img_exists and mask_exists:
            valid_count += 1
            print(f"  ✅ Satır {idx}: Her iki path geçerli")
        else:
            print(f"  ⚠️  Satır {idx}:")
            print(f"     Image: {'✅' if img_exists else '❌'} {str(img_path)[:80] if pd.notna(img_path) else 'None'}")
            print(f"     Mask:  {'✅' if mask_exists else '❌'} {str(mask_path)[:80] if pd.notna(mask_path) else 'None'}")
    
    if sample_size > 0:
        print(f"\n📈 Örnek Geçerlilik Oranı: {valid_count}/{sample_size} ({valid_count/sample_size*100:.1f}%)")
    else:
        print("  ⚠️  Örnek kontrol yapılamadı (DataFrame boş)")

# 6. Metadata kolonları
metadata_cols = [col for col in df.columns if col not in required_cols]
print(f"\n📋 Metadata Kolonları: {len(metadata_cols)} adet")
if metadata_cols:
    print(f"   İlk 10: {', '.join(metadata_cols[:10])}")

print("\n" + "="*60)
print("✅ CSV VALİDASYON TAMAMLANDI")
print("="*60)


In [ ]:
#HÜCRE 5: Model ve Processor İndirme
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
print("🔄 Model ve Processor indiriliyor (mit-b4)...")

# Processor
processor = SegformerImageProcessor.from_pretrained(
    CONFIG['model_name'], 
    do_reduce_labels=False,
    size={"height": CONFIG['img_size'], "width": CONFIG['img_size']}
)

# Model
model = SegformerForSemanticSegmentation.from_pretrained(
    CONFIG['model_name'],
    num_labels=CONFIG['num_classes'],
    id2label={0: "Background", 1: "Fake"},
    label2id={"Background": 0, "Fake": 1},
    ignore_mismatched_sizes=True,
    #use_safetensors=True
)
model.to(device)

print("✅ Model başarıyla yüklendi ve GPU'ya taşındı.")

# Dataset Objelerini Oluştur
train_dataset = FakeImageSegmentationDataset(train_df, processor, is_train=True)
val_dataset = FakeImageSegmentationDataset(val_df, processor, is_train=False)

In [ ]:
# =============================================================================
# HUCRE 6: ENHANCED METRICS (Confidence Tracking ile)
# =============================================================================
import evaluate
from scipy import ndimage

def compute_metrics_with_confidence(eval_pred):
    """
    Enhanced metrics including:
    - Standard IoU metrics (mean_iou, fake_iou, bg_iou)
    - Confidence metrics (per-pixel, boundary, sample-level)
    - Boundary analysis (edge vs interior IoU)
    """
    logits, labels = eval_pred
    
    # Convert to tensors
    logits_tensor = torch.from_numpy(logits)
    
    # Upsample logits to match label size
    logits_upsampled = torch.nn.functional.interpolate(
        logits_tensor, size=labels.shape[-2:],
        mode="bilinear", align_corners=False
    )
    
    # Get predictions and probabilities
    probs = torch.softmax(logits_upsampled, dim=1)
    preds = probs.argmax(dim=1).numpy()
    probs_np = probs.numpy()
    
    # === STANDARD IoU METRICS ===
    metric = evaluate.load("mean_iou")
    iou_metrics = metric.compute(
        predictions=preds,
        references=labels,
        num_labels=CONFIG['num_classes'],
        ignore_index=255,
        reduce_labels=False
    )
    
    # === CONFIDENCE METRICS ===
    valid_mask = labels != 255
    
    # 1. Per-pixel average confidence (mean of max softmax)
    max_probs = probs_np.max(axis=1)  # [B, H, W]
    avg_pixel_confidence = float(max_probs[valid_mask].mean()) if valid_mask.sum() > 0 else 0.0
    
    # 2. Boundary confidence (edge pixels only using sobel)
    boundary_confidences = []
    boundary_ious = []
    interior_ious = []
    sample_ious = []
    
    for i in range(len(labels)):
        # Detect edges using sobel on ground truth
        edges = np.abs(ndimage.sobel(labels[i].astype(float), axis=0)) + \
                np.abs(ndimage.sobel(labels[i].astype(float), axis=1))
        edge_mask = (edges > 0) & valid_mask[i]
        interior_mask = (edges == 0) & valid_mask[i]
        
        # Boundary confidence
        if edge_mask.sum() > 0:
            boundary_conf = max_probs[i][edge_mask].mean()
            boundary_confidences.append(float(boundary_conf))
            
            # Boundary IoU (Fake class only)
            pred_edge = preds[i][edge_mask]
            label_edge = labels[i][edge_mask]
            intersection = ((pred_edge == 1) & (label_edge == 1)).sum()
            union = ((pred_edge == 1) | (label_edge == 1)).sum()
            if union > 0:
                boundary_ious.append(float(intersection / union))
        
        # Interior IoU
        if interior_mask.sum() > 0:
            pred_int = preds[i][interior_mask]
            label_int = labels[i][interior_mask]
            intersection = ((pred_int == 1) & (label_int == 1)).sum()
            union = ((pred_int == 1) | (label_int == 1)).sum()
            if union > 0:
                interior_ious.append(float(intersection / union))
        
        # Per-sample IoU (Fake class)
        valid_pred = preds[i][valid_mask[i]]
        valid_label = labels[i][valid_mask[i]]
        intersection = ((valid_pred == 1) & (valid_label == 1)).sum()
        union = ((valid_pred == 1) | (valid_label == 1)).sum()
        if union > 0:
            sample_ious.append(float(intersection / union))
    
    return {
        # Standard metrics
        "mean_iou": float(iou_metrics["mean_iou"]),
        "accuracy": float(iou_metrics["mean_accuracy"]),
        "fake_iou": float(iou_metrics["per_category_iou"][1]) if len(iou_metrics["per_category_iou"]) > 1 else 0.0,
        "bg_iou": float(iou_metrics["per_category_iou"][0]),
        
        # Confidence metrics
        "avg_pixel_confidence": avg_pixel_confidence,
        "avg_boundary_confidence": float(np.mean(boundary_confidences)) if boundary_confidences else 0.0,
        "avg_sample_iou": float(np.mean(sample_ious)) if sample_ious else 0.0,
        
        # Boundary analysis
        "boundary_iou": float(np.mean(boundary_ious)) if boundary_ious else 0.0,
        "interior_iou": float(np.mean(interior_ious)) if interior_ious else 0.0,
    }

print("Enhanced Metrik fonksiyonu hazır:")
print("  - Standard: mean_iou, fake_iou, bg_iou, accuracy")
print("  - Confidence: pixel, boundary, sample-level")
print("  - Boundary Analysis: boundary_iou vs interior_iou")

In [ ]:
# =============================================================================
# HUCRE 6.5: CUSTOM TRAINER VE METRICS TRACKER CALLBACK
# =============================================================================
from transformers import TrainerCallback, Trainer
from tqdm.auto import tqdm
import time

# -----------------------------------------------------------------------------
# CUSTOM TRAINER: Custom Loss Function desteği
# -----------------------------------------------------------------------------
class SegformerTrainerWithCustomLoss(Trainer):
    """
    HuggingFace Trainer with custom loss function support
    Allows using DiceFocalLoss instead of default CrossEntropy
    """
    def __init__(self, *args, custom_loss_fn=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.custom_loss_fn = custom_loss_fn
    
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Upsample logits to match label size
        upsampled_logits = nn.functional.interpolate(
            logits, size=labels.shape[-2:],
            mode="bilinear", align_corners=False
        )
        
        # Use custom loss if provided
        if self.custom_loss_fn is not None:
            loss = self.custom_loss_fn(upsampled_logits, labels)
        else:
            # Default CrossEntropy
            loss = nn.functional.cross_entropy(
                upsampled_logits.view(-1, CONFIG['num_classes']),
                labels.view(-1).long(),
                ignore_index=255
            )
        
        return (loss, outputs) if return_outputs else loss


# -----------------------------------------------------------------------------
# METRICS TRACKER CALLBACK: Progress bar + History tracking + CSV export
# -----------------------------------------------------------------------------
class MetricsTrackerCallback(TrainerCallback):
    """
    Comprehensive callback for:
    1. Real-time progress bar with IoU metrics
    2. History tracking (all metrics per epoch)
    3. CSV export after each epoch
    4. Best model tracking
    """
    def __init__(self, data_dir, viz_dir):
        self.data_dir = data_dir
        self.viz_dir = viz_dir
        self.models_dir = models_dir  # Model kaydetme için (best_model ve last_model)
        
        # History tracking
        self.history = {
            'epoch': [],
            'train_loss': [],
            'val_loss': [],
            'val_mean_iou': [],
            'val_fake_iou': [],
            'val_bg_iou': [],
            'val_accuracy': [],
            'val_pixel_confidence': [],
            'val_boundary_confidence': [],
            'val_sample_iou': [],
            'val_boundary_iou': [],
            'val_interior_iou': [],
            'learning_rate': []
        }
        
        # Progress tracking
        self.train_pbar = None
        self.current_epoch = 0
        self.epoch_start_time = None
        
        # Best model tracking
        self.best_fake_iou = 0.0
        self.best_epoch = 0
    
    def on_train_begin(self, args, state, control, **kwargs):
        print(f"\n{'='*70}")
        print(f"EGITIM BASLIYOR")
        print(f"{'='*70}")
        print(f"  Total epochs: {int(args.num_train_epochs)}")
        print(f"  Total steps: {state.max_steps}")
        print(f"  Loss type: {CONFIG['loss_type']}")
        print(f"{'='*70}\n")
    
    def on_epoch_begin(self, args, state, control, **kwargs):
        self.current_epoch = int(state.epoch) if state.epoch else 0
        self.epoch_start_time = time.time()
        
        print(f"\n{'─'*70}")
        print(f"EPOCH {self.current_epoch + 1}/{int(args.num_train_epochs)}")
        print(f"{'─'*70}")
        
        # Calculate steps per epoch
        steps_per_epoch = max(1, state.max_steps // int(args.num_train_epochs))
        
        # Create progress bar
        self.train_pbar = tqdm(
            total=steps_per_epoch,
            desc=f"Training",
            position=0,
            leave=True,
            ncols=100
        )
    
    def on_step_end(self, args, state, control, **kwargs):
        if self.train_pbar:
            self.train_pbar.update(1)
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        if self.train_pbar and logs:
            # Update progress bar with current metrics
            postfix = {}
            
            if 'loss' in logs:
                postfix['loss'] = f"{logs['loss']:.4f}"
            
            if 'learning_rate' in logs:
                postfix['lr'] = f"{logs['learning_rate']:.2e}"
            
            # Show eval metrics if available
            if 'eval_mean_iou' in logs:
                postfix['mIoU'] = f"{logs['eval_mean_iou']:.4f}"
            if 'eval_fake_iou' in logs:
                postfix['fIoU'] = f"{logs['eval_fake_iou']:.4f}"
            
            self.train_pbar.set_postfix(postfix)
    
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """Called after each evaluation"""
        if self.train_pbar:
            self.train_pbar.close()
            self.train_pbar = None
        
        if metrics:
            epoch_num = self.current_epoch + 1
            epoch_time = time.time() - self.epoch_start_time if self.epoch_start_time else 0
            
            # Record history
            self.history['epoch'].append(epoch_num)
            self.history['train_loss'].append(metrics.get('train_loss', 0))
            self.history['val_loss'].append(metrics.get('eval_loss', 0))
            self.history['val_mean_iou'].append(metrics.get('eval_mean_iou', 0))
            self.history['val_fake_iou'].append(metrics.get('eval_fake_iou', 0))
            self.history['val_bg_iou'].append(metrics.get('eval_bg_iou', 0))
            self.history['val_accuracy'].append(metrics.get('eval_accuracy', 0))
            self.history['val_pixel_confidence'].append(metrics.get('eval_avg_pixel_confidence', 0))
            self.history['val_boundary_confidence'].append(metrics.get('eval_avg_boundary_confidence', 0))
            self.history['val_sample_iou'].append(metrics.get('eval_avg_sample_iou', 0))
            self.history['val_boundary_iou'].append(metrics.get('eval_boundary_iou', 0))
            self.history['val_interior_iou'].append(metrics.get('eval_interior_iou', 0))
            self.history['learning_rate'].append(state.log_history[-1].get('learning_rate', 0) if state.log_history else 0)
            
            # Track best model
            current_fake_iou = metrics.get('eval_fake_iou', 0)
            is_best = current_fake_iou > self.best_fake_iou
            if is_best:
                self.best_fake_iou = current_fake_iou
                self.best_epoch = epoch_num
                self._is_best_epoch = True  # Model kaydetme için flag
            else:
                self._is_best_epoch = False

            # Save models: best_model and last_model
            if self.models_dir:
                trainer = kwargs.get('trainer', None)
                if trainer is not None:
                    # Her epoch sonunda last_model'i kaydet
                    last_model_path = os.path.join(self.models_dir, "last_model")
                    trainer.save_model(last_model_path)
                    # Processor'ı kaydet (global processor değişkeninden)
                    try:
                        if 'processor' in globals():
                            processor.save_pretrained(last_model_path)
                    except Exception as e:
                        print(f"  ⚠️  Processor kaydedilemedi: {e}")
                    
                    # En iyi model ise best_model'i güncelle
                    if is_best:
                        best_model_path = os.path.join(self.models_dir, "best_model")
                        trainer.save_model(best_model_path)
                        try:
                            if 'processor' in globals():
                                processor.save_pretrained(best_model_path)
                        except Exception as e:
                            print(f"  ⚠️  Processor kaydedilemedi: {e}")
                        print(f"  💾 Best model kaydedildi: {best_model_path} (Fake IoU: {current_fake_iou:.4f})")
                    else:
                        print(f"  💾 Last model kaydedildi: {last_model_path} (Epoch {epoch_num})")
            
            # Print epoch summary
            print(f"\n  Epoch {epoch_num} Summary ({epoch_time:.1f}s):")
            print(f"  {'─'*50}")
            print(f"  Train Loss:        {metrics.get('train_loss', 0):.4f}")
            print(f"  Val Loss:          {metrics.get('eval_loss', 0):.4f}")
            print(f"  Val Mean IoU:      {metrics.get('eval_mean_iou', 0):.4f}")
            print(f"  Val Fake IoU:      {metrics.get('eval_fake_iou', 0):.4f} {'NEW BEST!' if is_best else ''}")
            print(f"  Val BG IoU:        {metrics.get('eval_bg_iou', 0):.4f}")
            print(f"  Val Accuracy:      {metrics.get('eval_accuracy', 0):.4f}")
            print(f"  {'─'*50}")
            print(f"  Pixel Confidence:  {metrics.get('eval_avg_pixel_confidence', 0):.4f}")
            print(f"  Boundary Conf:     {metrics.get('eval_avg_boundary_confidence', 0):.4f}")
            print(f"  Boundary IoU:      {metrics.get('eval_boundary_iou', 0):.4f}")
            print(f"  Interior IoU:      {metrics.get('eval_interior_iou', 0):.4f}")
            
            # Save history to CSV after each epoch
            self._save_history_csv()
    
    def _save_history_csv(self):
        """Export training history to CSV"""
        df = pd.DataFrame(self.history)
        csv_path = os.path.join(self.data_dir, 'training_history.csv')
        df.to_csv(csv_path, index=False)
    
    def on_train_end(self, args, state, control, **kwargs):
        print(f"\n{'='*70}")
        print(f"EGITIM TAMAMLANDI")
        print(f"{'='*70}")
        print(f"  Best Fake IoU: {self.best_fake_iou:.4f} (Epoch {self.best_epoch})")
        print(f"  Total Epochs:  {len(self.history['epoch'])}")
        print(f"  History saved: {os.path.join(self.data_dir, 'training_history.csv')}")
        print(f"{'='*70}\n")

print("Custom Trainer ve MetricsTrackerCallback hazir:")

In [ ]:
# =============================================================================
# HUCRE 7: TRAINING ARGUMENTS VE TRAINER SETUP
# =============================================================================
from transformers import TrainingArguments

# -----------------------------------------------------------------------------
# 1. Loss Function olustur (CONFIG'e gore)
# -----------------------------------------------------------------------------
if CONFIG['loss_type'] == 'dice_focal':
    focal_alpha = torch.tensor(CONFIG['focal_alpha']).float()
    custom_loss = DiceFocalLoss(
        dice_weight=CONFIG['dice_weight'],
        focal_weight=CONFIG['focal_weight'],
        focal_alpha=focal_alpha,
        focal_gamma=CONFIG['focal_gamma'],
        smooth=CONFIG['dice_smooth']
    )
    print(f"Loss Function: DiceFocalLoss")
    print(f"  Dice weight: {CONFIG['dice_weight']}, Focal weight: {CONFIG['focal_weight']}")
    print(f"  Focal gamma: {CONFIG['focal_gamma']}, Alpha: {CONFIG['focal_alpha']}")

elif CONFIG['loss_type'] == 'focal':
    focal_alpha = torch.tensor(CONFIG['focal_alpha']).float()
    custom_loss = FocalLoss(
        alpha=focal_alpha,
        gamma=CONFIG['focal_gamma']
    )
    print(f"Loss Function: FocalLoss")
    print(f"  Gamma: {CONFIG['focal_gamma']}, Alpha: {CONFIG['focal_alpha']}")

elif CONFIG['loss_type'] == 'dice':
    custom_loss = DiceLoss(smooth=CONFIG['dice_smooth'])
    print(f"Loss Function: DiceLoss")

else:  # crossentropy (default)
    custom_loss = None
    print(f"Loss Function: CrossEntropy (default)")

# -----------------------------------------------------------------------------
# 2. Training Arguments
# -----------------------------------------------------------------------------
training_args = TrainingArguments(
    output_dir=CONFIG['output_dir'],
    learning_rate=CONFIG['learning_rate'],
    num_train_epochs=CONFIG['num_epochs'],
    per_device_train_batch_size=CONFIG['batch_size'],
    per_device_eval_batch_size=CONFIG['batch_size'],
    gradient_accumulation_steps=CONFIG['gradient_accumulation'],
    weight_decay=CONFIG['weight_decay'],

    # Checkpoint strategy
    save_strategy="epoch",
    evaluation_strategy="epoch",  # eval_strategy yerine evaluation_strategy
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="fake_iou",
    greater_is_better=True,

    # GPU Optimizations (RTX 5080)
    fp16=True,
    tf32=True,
    
    # Windows compatibility
    dataloader_num_workers=0,
    dataloader_pin_memory=True,
    
    # Logging
    logging_steps=10,
    report_to="none",
    
    # Misc
    remove_unused_columns=False,
)

# -----------------------------------------------------------------------------
# 3. Callback olustur
# -----------------------------------------------------------------------------
metrics_callback = MetricsTrackerCallback(
    data_dir=CONFIG['data_dir'],
    viz_dir=CONFIG['viz_dir'],
    models_dir=CONFIG['models_dir']  # Model kaydetme için (best_model ve last_model)
)

# -----------------------------------------------------------------------------
# 4. Trainer olustur
# -----------------------------------------------------------------------------
trainer = SegformerTrainerWithCustomLoss(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics_with_confidence,
    callbacks=[metrics_callback],
    custom_loss_fn=custom_loss
)

print(f"\nTrainer hazir:")
print(f"  Custom Loss: {CONFIG['loss_type']}")
print(f"  Metrics Callback: Aktif")
print(f"  Output: {CONFIG['output_dir']}")

In [ ]:
# =============================================================================
# HUCRE 7.5: VISUALIZATION FUNCTIONS (Tam Paket)
# =============================================================================
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader

# Style setup
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# -----------------------------------------------------------------------------
# 1. TRAINING CURVES (3 panel)
# -----------------------------------------------------------------------------
def plot_training_curves(history, save_path):
    """
    Plot training curves: Loss, IoU, Learning Rate
    """
    epochs = history['epoch']
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Panel 1: Loss
    axes[0].plot(epochs, history['train_loss'], 'b-o', label='Train Loss', linewidth=2, markersize=6)
    axes[0].plot(epochs, history['val_loss'], 'r-s', label='Val Loss', linewidth=2, markersize=6)
    axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Loss', fontsize=12, fontweight='bold')
    axes[0].set_title('Training & Validation Loss', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)
    
    # Panel 2: IoU Metrics
    axes[1].plot(epochs, history['val_mean_iou'], 'g-o', label='Mean IoU', linewidth=2, markersize=6)
    axes[1].plot(epochs, history['val_fake_iou'], color='orange', marker='s', label='Fake IoU', linewidth=2, markersize=6)
    axes[1].plot(epochs, history['val_bg_iou'], color='purple', marker='^', label='BG IoU', linewidth=2, markersize=6)
    axes[1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('IoU Score', fontsize=12, fontweight='bold')
    axes[1].set_title('Validation IoU Metrics', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)
    axes[1].set_ylim([0, 1])
    
    # Panel 3: Learning Rate
    axes[2].plot(epochs, history['learning_rate'], 'purple', marker='o', linewidth=2, markersize=6)
    axes[2].set_xlabel('Epoch', fontsize=12, fontweight='bold')
    axes[2].set_ylabel('Learning Rate', fontsize=12, fontweight='bold')
    axes[2].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[2].set_yscale('log')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Training curves kaydedildi: {save_path}")


# -----------------------------------------------------------------------------
# 2. COMPREHENSIVE ANALYSIS DASHBOARD (8 panel)
# -----------------------------------------------------------------------------
def plot_comprehensive_analysis(history, save_path, version):
    """
    8-panel comprehensive dashboard
    """
    epochs = history['epoch']
    
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)
    
    # Panel 1: Loss Curves
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(epochs, history['train_loss'], 'b-o', label='Train', linewidth=2, markersize=4)
    ax1.plot(epochs, history['val_loss'], 'r-s', label='Val', linewidth=2, markersize=4)
    ax1.set_title('Loss Curves', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Panel 2: IoU Metrics
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(epochs, history['val_mean_iou'], 'g-o', label='Mean IoU', linewidth=2)
    ax2.plot(epochs, history['val_fake_iou'], color='orange', marker='s', label='Fake IoU', linewidth=2)
    ax2.set_title('IoU Performance', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('IoU')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim([0, 1])
    
    # Panel 3: Learning Rate
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.plot(epochs, history['learning_rate'], 'purple', marker='o', linewidth=2)
    ax3.set_title('Learning Rate', fontsize=12, fontweight='bold')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('LR')
    ax3.set_yscale('log')
    ax3.grid(True, alpha=0.3)
    
    # Panel 4: Loss Gap (Overfitting indicator)
    ax4 = fig.add_subplot(gs[1, 0])
    loss_gap = [v - t for t, v in zip(history['train_loss'], history['val_loss'])]
    ax4.plot(epochs, loss_gap, 'darkred', marker='o', linewidth=2)
    ax4.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax4.set_title('Loss Gap (Val - Train)', fontsize=12, fontweight='bold')
    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('Gap')
    ax4.grid(True, alpha=0.3)
    ax4.fill_between(epochs, 0, loss_gap, alpha=0.3, color='red')
    
    # Panel 5: Confidence Metrics
    ax5 = fig.add_subplot(gs[1, 1])
    ax5.plot(epochs, history['val_pixel_confidence'], 'cyan', marker='o', label='Pixel Conf', linewidth=2)
    ax5.plot(epochs, history['val_boundary_confidence'], 'magenta', marker='s', label='Boundary Conf', linewidth=2)
    ax5.set_title('Confidence Metrics', fontsize=12, fontweight='bold')
    ax5.set_xlabel('Epoch')
    ax5.set_ylabel('Confidence')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    ax5.set_ylim([0, 1])
    
    # Panel 6: Boundary vs Interior IoU
    ax6 = fig.add_subplot(gs[1, 2])
    ax6.plot(epochs, history['val_boundary_iou'], 'darkblue', marker='o', label='Boundary', linewidth=2)
    ax6.plot(epochs, history['val_interior_iou'], 'darkgreen', marker='s', label='Interior', linewidth=2)
    ax6.set_title('Boundary vs Interior IoU', fontsize=12, fontweight='bold')
    ax6.set_xlabel('Epoch')
    ax6.set_ylabel('IoU')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    ax6.set_ylim([0, 1])
    
    # Panel 7: Metrics Table
    ax7 = fig.add_subplot(gs[2, :])
    ax7.axis('off')
    
    table_data = []
    for i, epoch in enumerate(epochs):
        row = [
            f"{epoch}",
            f"{history['train_loss'][i]:.4f}",
            f"{history['val_loss'][i]:.4f}",
            f"{history['val_mean_iou'][i]:.4f}",
            f"{history['val_fake_iou'][i]:.4f}",
            f"{history['val_accuracy'][i]:.4f}",
            f"{history['learning_rate'][i]:.2e}"
        ]
        table_data.append(row)
    
    table = ax7.table(
        cellText=table_data,
        colLabels=['Epoch', 'Train Loss', 'Val Loss', 'Mean IoU', 'Fake IoU', 'Accuracy', 'LR'],
        cellLoc='center',
        loc='center',
        bbox=[0, 0, 1, 1]
    )
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 1.5)
    
    # Header style
    for i in range(7):
        table[(0, i)].set_facecolor('#4CAF50')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    # Highlight best epoch
    if history['val_fake_iou']:
        best_idx = history['val_fake_iou'].index(max(history['val_fake_iou']))
        for i in range(7):
            table[(best_idx + 1, i)].set_facecolor('#FFD700')
    
    fig.suptitle(f'Comprehensive Training Analysis - Version {version}', fontsize=16, fontweight='bold', y=0.98)
    
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Comprehensive analysis kaydedildi: {save_path}")


# -----------------------------------------------------------------------------
# 3. SEGMENTATION SAMPLES GRID
# -----------------------------------------------------------------------------
def plot_segmentation_samples(model, val_dataset, save_path, num_samples=8, device='cuda'):
    """
    Visualize segmentation predictions in grid
    Shows: Original | Ground Truth | Prediction | Overlay
    """
    model.eval()
    
    # Randomly select samples
    indices = np.random.choice(len(val_dataset), size=min(num_samples, len(val_dataset)), replace=False)
    
    fig, axes = plt.subplots(num_samples, 4, figsize=(16, 4*num_samples))
    
    with torch.no_grad():
        for idx, sample_idx in enumerate(indices):
            sample = val_dataset[int(sample_idx)]
            
            pixel_values = sample['pixel_values'].unsqueeze(0).to(device)
            gt_mask = sample['labels'].cpu().numpy()
            
            # Predict
            outputs = model(pixel_values=pixel_values)
            logits = outputs.logits
            
            # Upsample and get prediction
            upsampled = torch.nn.functional.interpolate(
                logits, size=gt_mask.shape,
                mode='bilinear', align_corners=False
            )
            pred_mask = upsampled.argmax(dim=1).squeeze().cpu().numpy()
            
            # Denormalize image
            img_array = pixel_values.squeeze().cpu().permute(1, 2, 0).numpy()
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img_array = img_array * std + mean
            img_array = np.clip(img_array, 0, 1)
            
            # Column 1: Original Image
            axes[idx, 0].imshow(img_array)
            axes[idx, 0].set_title('Original', fontweight='bold')
            axes[idx, 0].axis('off')
            
            # Column 2: Ground Truth
            axes[idx, 1].imshow(gt_mask, cmap='RdYlBu', vmin=0, vmax=1)
            axes[idx, 1].set_title('Ground Truth', fontweight='bold')
            axes[idx, 1].axis('off')
            
            # Column 3: Prediction
            axes[idx, 2].imshow(pred_mask, cmap='RdYlBu', vmin=0, vmax=1)
            axes[idx, 2].set_title('Prediction', fontweight='bold')
            axes[idx, 2].axis('off')
            
            # Column 4: Overlay
            overlay = img_array.copy()
            # Red for fake regions (prediction)
            fake_mask = pred_mask == 1
            overlay[fake_mask, 0] = np.clip(overlay[fake_mask, 0] + 0.5, 0, 1)
            overlay[fake_mask, 1] = overlay[fake_mask, 1] * 0.5
            overlay[fake_mask, 2] = overlay[fake_mask, 2] * 0.5
            axes[idx, 3].imshow(overlay)
            axes[idx, 3].set_title('Overlay (Red=Fake)', fontweight='bold')
            axes[idx, 3].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Segmentation samples kaydedildi: {save_path}")


# -----------------------------------------------------------------------------
# 4. IoU DISTRIBUTION
# -----------------------------------------------------------------------------
def plot_iou_distribution(history, save_path):
    """IoU distribution over epochs"""
    epochs = history['epoch']
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart: IoU comparison
    x = np.arange(len(epochs))
    width = 0.25
    
    axes[0].bar(x - width, history['val_fake_iou'], width, label='Fake IoU', color='orange', alpha=0.8)
    axes[0].bar(x, history['val_bg_iou'], width, label='BG IoU', color='blue', alpha=0.8)
    axes[0].bar(x + width, history['val_mean_iou'], width, label='Mean IoU', color='green', alpha=0.8)
    axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('IoU Score', fontsize=12, fontweight='bold')
    axes[0].set_title('IoU Comparison by Epoch', fontsize=14, fontweight='bold')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(epochs)
    axes[0].legend()
    axes[0].set_ylim([0, 1])
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Box plot: Boundary vs Interior
    boundary_data = history['val_boundary_iou']
    interior_data = history['val_interior_iou']
    
    axes[1].boxplot([boundary_data, interior_data], labels=['Boundary IoU', 'Interior IoU'])
    axes[1].set_ylabel('IoU Score', fontsize=12, fontweight='bold')
    axes[1].set_title('Boundary vs Interior IoU Distribution', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"IoU distribution kaydedildi: {save_path}")


# -----------------------------------------------------------------------------
# 5. CONFIDENCE DISTRIBUTION
# -----------------------------------------------------------------------------
def plot_confidence_analysis(history, save_path):
    """Confidence analysis plot"""
    epochs = history['epoch']
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Line plot: Confidence over epochs
    axes[0].plot(epochs, history['val_pixel_confidence'], 'b-o', label='Pixel Confidence', linewidth=2)
    axes[0].plot(epochs, history['val_boundary_confidence'], 'r-s', label='Boundary Confidence', linewidth=2)
    axes[0].fill_between(epochs, history['val_pixel_confidence'], history['val_boundary_confidence'], alpha=0.3)
    axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Confidence', fontsize=12, fontweight='bold')
    axes[0].set_title('Confidence Over Training', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].set_ylim([0, 1])
    axes[0].grid(True, alpha=0.3)
    
    # Scatter: Confidence vs IoU
    axes[1].scatter(history['val_pixel_confidence'], history['val_fake_iou'], 
                   c=epochs, cmap='viridis', s=100, alpha=0.8)
    axes[1].set_xlabel('Pixel Confidence', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Fake IoU', fontsize=12, fontweight='bold')
    axes[1].set_title('Confidence vs IoU (Color=Epoch)', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    cbar = plt.colorbar(axes[1].collections[0], ax=axes[1])
    cbar.set_label('Epoch')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Confidence analysis kaydedildi: {save_path}")


# -----------------------------------------------------------------------------
# 6. ROC/PR CURVES (Segmentation için threshold-based)
# -----------------------------------------------------------------------------
def plot_roc_pr_curves(all_labels, all_confidences, save_path):
    """
    Segmentation için threshold-based ROC/PR curves
    """
    from sklearn.metrics import roc_curve, precision_recall_curve, auc
    
    # Binary classification için: Fake class = 1
    y_true = (np.array(all_labels) == 1).astype(int)
    y_scores = np.array(all_confidences)
    
    # ROC Curve
    fpr, tpr, roc_thresholds = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    # PR Curve
    precision, recall, pr_thresholds = precision_recall_curve(y_true, y_scores)
    ap_score = auc(recall, precision)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # ROC Curve
    axes[0].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    axes[0].set_xlim([0.0, 1.0])
    axes[0].set_ylim([0.0, 1.05])
    axes[0].set_xlabel('False Positive Rate', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('True Positive Rate', fontsize=12, fontweight='bold')
    axes[0].set_title('ROC Curve', fontsize=14, fontweight='bold')
    axes[0].legend(loc="lower right")
    axes[0].grid(True, alpha=0.3)
    
    # PR Curve
    axes[1].plot(recall, precision, color='darkblue', lw=2, label=f'PR curve (AP = {ap_score:.4f})')
    axes[1].set_xlim([0.0, 1.0])
    axes[1].set_ylim([0.0, 1.05])
    axes[1].set_xlabel('Recall', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Precision', fontsize=12, fontweight='bold')
    axes[1].set_title('Precision-Recall Curve', fontsize=14, fontweight='bold')
    axes[1].legend(loc="lower left")
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"ROC/PR curves kaydedildi: {save_path}")
    
    # Export data
    export_roc_pr_curve_data_csv(fpr, tpr, precision, recall, roc_thresholds, 
                                 save_path.replace('.png', '_data.csv'))
    
    return fpr, tpr, precision, recall, roc_auc, ap_score


# -----------------------------------------------------------------------------
# 7. CONFIDENCE THRESHOLD ANALYSIS
# -----------------------------------------------------------------------------
def plot_confidence_threshold_analysis(all_labels, all_confidences, all_predictions, save_path):
    """
    Farklı threshold'larda performans analizi
    """
    thresholds = np.arange(0.1, 1.0, 0.05)
    
    # Calculate metrics for each threshold
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    
    for threshold in thresholds:
        threshold_preds = (all_confidences >= threshold).astype(int)
        cm = confusion_matrix(all_labels, threshold_preds)
        if cm.size == 4:
            tn, fp, fn, tp = cm.ravel()
            acc = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
            prec = tp / (tp + fp) if (tp + fp) > 0 else 0
            rec = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
        else:
            acc = prec = rec = f1 = 0
        
        accuracies.append(acc)
        precisions.append(prec)
        recalls.append(rec)
        f1_scores.append(f1)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Metrics vs Threshold
    axes[0].plot(thresholds, accuracies, 'b-o', label='Accuracy', linewidth=2, markersize=4)
    axes[0].plot(thresholds, precisions, 'r-s', label='Precision', linewidth=2, markersize=4)
    axes[0].plot(thresholds, recalls, 'g-^', label='Recall', linewidth=2, markersize=4)
    axes[0].plot(thresholds, f1_scores, 'm-d', label='F1-Score', linewidth=2, markersize=4)
    axes[0].set_xlabel('Confidence Threshold', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Metric Value', fontsize=12, fontweight='bold')
    axes[0].set_title('Performance vs Confidence Threshold', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].set_ylim([0, 1])
    
    # Optimal threshold (max F1)
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx]
    axes[0].axvline(x=optimal_threshold, color='orange', linestyle='--', linewidth=2, 
                   label=f'Optimal (F1={f1_scores[optimal_idx]:.3f})')
    axes[0].legend()
    
    # Precision-Recall Trade-off
    axes[1].plot(recalls, precisions, 'b-o', linewidth=2, markersize=4)
    axes[1].scatter([recalls[optimal_idx]], [precisions[optimal_idx]], 
                   color='red', s=200, zorder=5, label=f'Optimal (t={optimal_threshold:.2f})')
    axes[1].set_xlabel('Recall', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Precision', fontsize=12, fontweight='bold')
    axes[1].set_title('Precision-Recall Trade-off', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    axes[1].set_ylim([0, 1])
    axes[1].set_xlim([0, 1])
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Confidence threshold analysis kaydedildi: {save_path}")
    print(f"Optimal threshold: {optimal_threshold:.3f} (F1={f1_scores[optimal_idx]:.3f})")
    
    return optimal_threshold, thresholds, accuracies, precisions, recalls, f1_scores


# -----------------------------------------------------------------------------
# 8. CALIBRATION CURVES
# -----------------------------------------------------------------------------
def plot_calibration_curves(all_labels, all_confidences, save_path, n_bins=10):
    """
    Model calibration analizi
    """
    from sklearn.calibration import calibration_curve
    
    y_true = (np.array(all_labels) == 1).astype(int)
    y_scores = np.array(all_confidences)
    
    # Calibration curve
    fraction_of_positives, mean_predicted_value = calibration_curve(
        y_true, y_scores, n_bins=n_bins, strategy='uniform'
    )
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Calibration plot
    axes[0].plot(mean_predicted_value, fraction_of_positives, "s-", label='Model', linewidth=2)
    axes[0].plot([0, 1], [0, 1], "k--", label='Perfectly Calibrated', linewidth=2)
    axes[0].set_xlabel('Mean Predicted Probability', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Fraction of Positives', fontsize=12, fontweight='bold')
    axes[0].set_title('Calibration Plot', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xlim([0, 1])
    axes[0].set_ylim([0, 1])
    
    # Confidence distribution
    axes[1].hist(y_scores, bins=50, alpha=0.7, edgecolor='black')
    axes[1].set_xlabel('Confidence Score', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
    axes[1].set_title('Confidence Distribution', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Calibration curves kaydedildi: {save_path}")
    
    return fraction_of_positives, mean_predicted_value


print("Visualization fonksiyonlari hazir:")
print("  - plot_training_curves()")
print("  - plot_comprehensive_analysis()")
print("  - plot_segmentation_samples()")
print("  - plot_iou_distribution()")
print("  - plot_confidence_analysis()")
print("  - plot_roc_pr_curves()")
print("  - plot_confidence_threshold_analysis()")
print("  - plot_calibration_curves()")

In [ ]:
# =============================================================================
# HUCRE 7.6: CSV EXPORT FUNCTIONS (SegFormer Analytics)
# =============================================================================
"""
CSV export fonksiyonları - SegFormer segmentation analizi için
"""

import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from scipy import ndimage

def export_confusion_matrix_csv(all_labels, all_predictions, save_path, class_names=['Background', 'Fake']):
    """
    Confusion matrix'i CSV olarak export et (counts & percentages)
    """
    cm = confusion_matrix(all_labels, all_predictions)
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Counts DataFrame
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
    cm_df.index.name = 'True Label'
    cm_df.columns.name = 'Predicted Label'
    
    # Percentages DataFrame
    cm_norm_df = pd.DataFrame(cm_norm * 100, index=class_names, columns=class_names)
    cm_norm_df.index.name = 'True Label'
    cm_norm_df.columns.name = 'Predicted Label'
    
    # Save counts
    counts_path = save_path.replace('.csv', '_counts.csv')
    cm_df.to_csv(counts_path)
    print(f"✅ Confusion matrix (counts) saved: {counts_path}")
    
    # Save percentages
    percent_path = save_path.replace('.csv', '_percentages.csv')
    cm_norm_df.to_csv(percent_path)
    print(f"✅ Confusion matrix (percentages) saved: {percent_path}")
    
    return cm_df, cm_norm_df


def export_classification_report_csv(all_labels, all_predictions, save_path, class_names=['Background', 'Fake']):
    """
    Per-class metrics (precision, recall, f1) CSV export
    """
    report_dict = classification_report(
        all_labels, all_predictions,
        target_names=class_names,
        output_dict=True
    )
    
    # Per-class metrics
    per_class_data = []
    for class_name in class_names:
        if class_name in report_dict:
            per_class_data.append({
                'class': class_name,
                'precision': report_dict[class_name]['precision'],
                'recall': report_dict[class_name]['recall'],
                'f1_score': report_dict[class_name]['f1-score'],
                'support': report_dict[class_name]['support']
            })
    
    # Overall metrics
    per_class_data.append({
        'class': 'weighted_avg',
        'precision': report_dict['weighted avg']['precision'],
        'recall': report_dict['weighted avg']['recall'],
        'f1_score': report_dict['weighted avg']['f1-score'],
        'support': report_dict['weighted avg']['support']
    })
    
    per_class_data.append({
        'class': 'macro_avg',
        'precision': report_dict['macro avg']['precision'],
        'recall': report_dict['macro avg']['recall'],
        'f1_score': report_dict['macro avg']['f1-score'],
        'support': report_dict['macro avg']['support']
    })
    
    df = pd.DataFrame(per_class_data)
    df.to_csv(save_path, index=False)
    print(f"✅ Classification report saved: {save_path}")
    
    return df


def export_per_class_metrics_csv(iou_metrics, save_path):
    """
    Background vs Fake IoU karşılaştırması
    """
    metrics_data = {
        'metric': ['mean_iou', 'background_iou', 'fake_iou', 'accuracy'],
        'value': [
            iou_metrics.get('mean_iou', 0),
            iou_metrics.get('bg_iou', 0),
            iou_metrics.get('fake_iou', 0),
            iou_metrics.get('accuracy', 0)
        ]
    }
    
    df = pd.DataFrame(metrics_data)
    df.to_csv(save_path, index=False)
    print(f"✅ Per-class metrics saved: {save_path}")
    
    return df


def export_confidence_statistics_csv(all_confidences, save_path):
    """
    Confidence dağılım istatistikleri
    """
    conf_arr = np.array(all_confidences)
    
    stats = {
        'statistic': ['mean', 'std', 'min', '25th_percentile', 'median', '75th_percentile', 'max'],
        'value': [
            float(np.mean(conf_arr)),
            float(np.std(conf_arr)),
            float(np.min(conf_arr)),
            float(np.percentile(conf_arr, 25)),
            float(np.median(conf_arr)),
            float(np.percentile(conf_arr, 75)),
            float(np.max(conf_arr))
        ]
    }
    
    df = pd.DataFrame(stats)
    df.to_csv(save_path, index=False)
    print(f"✅ Confidence statistics saved: {save_path}")
    
    return df


def export_roc_pr_curve_data_csv(fpr, tpr, precision, recall, thresholds, save_path):
    """
    ROC/PR curve verileri CSV export
    """
    # ROC data
    roc_data = {
        'fpr': fpr,
        'tpr': tpr,
        'threshold': list(thresholds) + [np.nan] * (len(fpr) - len(thresholds)) if len(thresholds) < len(fpr) else thresholds
    }
    roc_df = pd.DataFrame(roc_data)
    roc_path = save_path.replace('.csv', '_roc.csv')
    roc_df.to_csv(roc_path, index=False)
    print(f"✅ ROC curve data saved: {roc_path}")
    
    # PR data
    pr_data = {
        'precision': precision,
        'recall': recall,
        'threshold': list(thresholds) + [np.nan] * (len(precision) - len(thresholds)) if len(thresholds) < len(precision) else thresholds
    }
    pr_df = pd.DataFrame(pr_data)
    pr_path = save_path.replace('.csv', '_pr.csv')
    pr_df.to_csv(pr_path, index=False)
    print(f"✅ PR curve data saved: {pr_path}")
    
    return roc_df, pr_df


def export_confidence_threshold_analysis_csv(all_labels, all_confidences, all_predictions, save_path, thresholds=None):
    """
    Farklı confidence threshold'larında performans analizi
    """
    if thresholds is None:
        thresholds = np.arange(0.1, 1.0, 0.05)
    
    results = []
    for threshold in thresholds:
        # Threshold'a göre prediction
        threshold_preds = (all_confidences >= threshold).astype(int)
        
        # Metrics
        cm = confusion_matrix(all_labels, threshold_preds)
        if cm.size == 4:
            tn, fp, fn, tp = cm.ravel()
            accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        else:
            accuracy = precision = recall = f1 = 0
        
        results.append({
            'threshold': threshold,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'true_positives': int(tp) if cm.size == 4 else 0,
            'true_negatives': int(tn) if cm.size == 4 else 0,
            'false_positives': int(fp) if cm.size == 4 else 0,
            'false_negatives': int(fn) if cm.size == 4 else 0
        })
    
    df = pd.DataFrame(results)
    df.to_csv(save_path, index=False)
    print(f"✅ Confidence threshold analysis saved: {save_path}")
    
    return df


def export_spatial_analysis_csv(all_labels, all_predictions, all_metadata, save_path):
    """
    Region size, aspect ratio analizi (metadata'dan)
    """
    # Metadata'dan region bilgilerini topla
    spatial_data = []
    
    for i, metadata in enumerate(all_metadata):
        if metadata:
            spatial_data.append({
                'idx': i,
                'true_label': all_labels[i],
                'predicted_label': all_predictions[i],
                'correct': int(all_labels[i] == all_predictions[i]),
                'dataset': metadata.get('dataset', 'unknown'),
                'area_ratio': metadata.get('area_ratio', None),
                'sem_magnitude': metadata.get('sem_magnitude', 'unknown'),
            })
    
    if spatial_data:
        df = pd.DataFrame(spatial_data)
        df.to_csv(save_path, index=False)
        print(f"✅ Spatial analysis saved: {save_path}")
        return df
    else:
        print(f"⚠️ No spatial metadata available")
        return None


def export_error_analysis_csv(all_labels, all_predictions, all_confidences, all_metadata, save_path):
    """
    False positive/negative detayları
    """
    error_data = []
    
    for i in range(len(all_labels)):
        true_label = all_labels[i]
        pred_label = all_predictions[i]
        confidence = all_confidences[i]
        metadata = all_metadata[i] if i < len(all_metadata) else {}
        
        if true_label != pred_label:
            error_type = 'false_positive' if pred_label == 1 and true_label == 0 else 'false_negative'
            
            error_data.append({
                'idx': i,
                'error_type': error_type,
                'true_label': true_label,
                'predicted_label': pred_label,
                'confidence': confidence,
                'dataset': metadata.get('dataset', 'unknown'),
                'area_ratio': metadata.get('area_ratio', None),
                'sem_magnitude': metadata.get('sem_magnitude', 'unknown'),
            })
    
    if error_data:
        df = pd.DataFrame(error_data)
        df.to_csv(save_path, index=False)
        print(f"✅ Error analysis saved: {save_path} ({len(error_data)} errors)")
        return df
    else:
        print(f"✅ No errors found!")
        return pd.DataFrame()


print("✅ CSV export fonksiyonları hazır:")
print("  - export_confusion_matrix_csv()")
print("  - export_classification_report_csv()")
print("  - export_per_class_metrics_csv()")
print("  - export_confidence_statistics_csv()")
print("  - export_roc_pr_curve_data_csv()")
print("  - export_confidence_threshold_analysis_csv()")
print("  - export_spatial_analysis_csv()")
print("  - export_error_analysis_csv()")


In [ ]:
# =============================================================================
# HUCRE 8: EGITIM BASLAT VE POST-TRAINING ISLEMLERI
# =============================================================================
print(f"\n{'='*70}")
print(f"EGITIM BASLATILIYOR - {CONFIG['num_epochs']} Epoch")
print(f"{'='*70}")
print(f"  Model: {CONFIG['model_name']}")
print(f"  Loss: {CONFIG['loss_type']}")
print(f"  Output: {CONFIG['output_dir']}")
print(f"{'='*70}\n")

# 1. EGITIM
trainer.train()

# 2. POST-TRAINING VISUALIZATIONS
print("\n" + "="*70)
print("POST-TRAINING VISUALIZATIONS")
print("="*70 + "\n")

# History'yi callback'ten al
history = metrics_callback.history

# 2.1 Training Curves
curves_path = os.path.join(CONFIG['viz_dir'], 'training_curves.png')
plot_training_curves(history, curves_path)

# 2.2 Comprehensive Analysis Dashboard
analysis_path = os.path.join(CONFIG['viz_dir'], 'comprehensive_training_analysis.png')
plot_comprehensive_analysis(history, analysis_path, CONFIG['version'])

# 2.3 Segmentation Samples
samples_path = os.path.join(CONFIG['viz_dir'], 'segmentation_samples.png')
plot_segmentation_samples(model, val_dataset, samples_path, num_samples=8, device=device)

# 2.4 IoU Distribution
iou_dist_path = os.path.join(CONFIG['viz_dir'], 'iou_distribution.png')
plot_iou_distribution(history, iou_dist_path)

# 2.5 Confidence Analysis
conf_path = os.path.join(CONFIG['viz_dir'], 'confidence_analysis.png')
plot_confidence_analysis(history, conf_path)

# 3. MODEL KAYDET
print("\n" + "="*70)
print("MODEL KAYDEDILIYOR")
print("="*70)

final_model_path = os.path.join(CONFIG['models_dir'], "final_best_model")
trainer.save_model(final_model_path)
processor.save_pretrained(final_model_path)

print(f"  Model: {final_model_path}")
print(f"  History: {os.path.join(CONFIG['data_dir'], 'training_history.csv')}")
print(f"  Gorseller: {CONFIG['viz_dir']}")

# 4. OZET RAPOR
print("\n" + "="*70)
print(f"EGITIM TAMAMLANDI - Version {CONFIG['version']}")
print("="*70)
print(f"  Best Fake IoU:     {metrics_callback.best_fake_iou:.4f}")
print(f"  Best Epoch:        {metrics_callback.best_epoch}")
print(f"  Total Epochs:      {len(history['epoch'])}")
print(f"  Final Mean IoU:    {history['val_mean_iou'][-1]:.4f}")
print(f"  Final Accuracy:    {history['val_accuracy'][-1]:.4f}")
print("="*70)

In [ ]:
# =============================================================================
# HUCRE 9: EGITIM OZETI VE KAYIT KONTROLU
# =============================================================================
# Bu hucre, egitim tamamlandiktan sonra ozet bilgi gosterir

print("="*70)
print("EGITIM TAMAMLANDI - DOSYA KONTROLU")
print("="*70)

# CSV kontrolu
csv_path = os.path.join(CONFIG['data_dir'], 'training_history.csv')
if os.path.exists(csv_path):
    history_df = pd.read_csv(csv_path)
    print(f"\nTraining History ({len(history_df)} epoch):")
    print(history_df[['epoch', 'val_mean_iou', 'val_fake_iou', 'val_accuracy']].to_string(index=False))
else:
    print("Henuz egitim yapilmadi!")

# Gorsel kontrolu
print(f"\nGorseller ({CONFIG['viz_dir']}):")
for f in os.listdir(CONFIG['viz_dir']) if os.path.exists(CONFIG['viz_dir']) else []:
    print(f"  - {f}")

# Model kontrolu
print(f"\nModeller ({CONFIG['models_dir']}):")
for f in os.listdir(CONFIG['models_dir']) if os.path.exists(CONFIG['models_dir']) else []:
    print(f"  - {f}")

In [ ]:
# =============================================================================
# HUCRE 10.5: VALIDATION SET CSV EXPORTS & ADVANCED VISUALIZATIONS
# =============================================================================
# Bu hücre validation set analizinden sonra CSV export'ları ve gelişmiş görselleştirmeler oluşturur

print("\n" + "="*70)
print("VALIDATION SET CSV EXPORTS & ADVANCED VISUALIZATIONS")
print("="*70)

# Önce validation set'i tekrar değerlendir (eğer yukarıdaki hücre çalıştırılmadıysa)
if 'all_preds' not in locals() or 'all_labels' not in locals():
    print("⚠️ Validation set analizi yapılmamış. Önce HUCRE 10'u çalıştırın.")
    print("   Validation set'i değerlendirmek için metadata ve confidence toplama eklenmiş versiyonu kullanın.")
else:
    # 1. Confusion Matrix
    cm_csv_path = os.path.join(CONFIG['data_dir'], 'confusion_matrix.csv')
    export_confusion_matrix_csv(all_labels, all_preds, cm_csv_path)
    
    # 2. Classification Report
    class_report_path = os.path.join(CONFIG['data_dir'], 'classification_report.csv')
    export_classification_report_csv(all_labels, all_preds, class_report_path)
    
    # 3. Per-class Metrics
    if 'iou_fake' in locals() and 'iou_bg' in locals() and 'mean_iou' in locals() and 'accuracy' in locals():
        iou_metrics = {
            'mean_iou': mean_iou,
            'bg_iou': iou_bg,
            'fake_iou': iou_fake,
            'accuracy': accuracy
        }
        per_class_path = os.path.join(CONFIG['data_dir'], 'per_class_metrics.csv')
        export_per_class_metrics_csv(iou_metrics, per_class_path)
    
    # 4. Confidence Statistics (if available)
    if 'all_confidences' in locals() and len(all_confidences) > 0:
        conf_stats_path = os.path.join(CONFIG['data_dir'], 'confidence_statistics.csv')
        export_confidence_statistics_csv(all_confidences, conf_stats_path)
        
        # 5. Confidence Threshold Analysis
        threshold_path = os.path.join(CONFIG['data_dir'], 'confidence_threshold_analysis.csv')
        export_confidence_threshold_analysis_csv(all_labels, all_confidences, all_preds, threshold_path)
    
    # 6. Error Analysis (if metadata available)
    if 'metadata_list' in locals() and len(metadata_list) > 0:
        error_analysis_path = os.path.join(CONFIG['data_dir'], 'error_analysis.csv')
        export_error_analysis_csv(all_labels, all_preds, all_confidences if 'all_confidences' in locals() else [], metadata_list, error_analysis_path)
        
        # 7. Spatial Analysis
        spatial_path = os.path.join(CONFIG['data_dir'], 'spatial_analysis.csv')
        export_spatial_analysis_csv(all_labels, all_preds, metadata_list, spatial_path)
    
    # =============================================================================
    # ADVANCED VISUALIZATIONS (if confidences available)
    # =============================================================================
    if 'all_confidences' in locals() and len(all_confidences) > 0:
        print("\n" + "="*70)
        print("ADVANCED VISUALIZATIONS")
        print("="*70)
        
        # 1. ROC/PR Curves
        roc_pr_path = os.path.join(CONFIG['viz_dir'], 'roc_pr_curves.png')
        plot_roc_pr_curves(all_labels, all_confidences, roc_pr_path)
        
        # 2. Confidence Threshold Analysis
        threshold_path = os.path.join(CONFIG['viz_dir'], 'confidence_threshold_analysis.png')
        plot_confidence_threshold_analysis(all_labels, all_confidences, all_preds, threshold_path)
        
        # 3. Calibration Curves
        calibration_path = os.path.join(CONFIG['viz_dir'], 'calibration_curves.png')
        plot_calibration_curves(all_labels, all_confidences, calibration_path)
        
        print("✅ Advanced visualizations completed!")
    
    # =============================================================================
    # DATASET ANALYSIS (if metadata available)
    # =============================================================================
    if 'metadata_list' in locals() and len(metadata_list) > 0:
        print("\n" + "="*70)
        print("DATASET ANALYSIS")
        print("="*70)
        
        # Dataset performance
        dataset_path = os.path.join(CONFIG['viz_dir'], 'dataset_performance.png')
        analyze_dataset_performance(all_labels, all_preds, metadata_list, dataset_path)
        
        # Semantic magnitude analysis
        magnitude_path = os.path.join(CONFIG['viz_dir'], 'semantic_magnitude_analysis.png')
        analyze_by_sem_magnitude(all_labels, all_preds, metadata_list, magnitude_path)
        
        print("✅ Dataset analysis completed!")
    
    print("\n" + "="*70)
    print("✅ Tüm CSV export'ları ve görselleştirmeler tamamlandı!")
    print(f"📁 CSV dosyaları: {CONFIG['data_dir']}")
    print(f"📊 Görseller: {CONFIG['viz_dir']}")
    print("="*70)


In [ ]:
# =============================================================================
# HUCRE 10: VALIDATION SET CONFUSION MATRIX (Opsiyonel)
# =============================================================================
# Bu hucre egitimden sonra detayli confusion matrix analizi yapar

import seaborn as sns
from sklearn.metrics import confusion_matrix
from tqdm.auto import tqdm
from torch.utils.data import DataLoader

print("VALIDATION SET ANALIZI BASLIYOR...")

# Validation loader
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
model.eval()

preds_list = []
labels_list = []

print("Validation verisi taranıyor...")
with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating"):
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].cpu().numpy()
        
        outputs = model(pixel_values=pixel_values)
        logits = torch.nn.functional.interpolate(
            outputs.logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
        )
        preds = logits.argmax(dim=1).cpu().numpy()
        
        # Sadece gecerli pikselleri al (255 olmayanlar)
        valid_mask = labels != 255
        preds_list.append(preds[valid_mask])
        labels_list.append(labels[valid_mask])

# Tek bir array yap
all_preds = np.concatenate(preds_list)
all_labels = np.concatenate(labels_list)

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Grafik
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Raw counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Background', 'Fake'], yticklabels=['Background', 'Fake'])
axes[0].set_title("Confusion Matrix (Raw Counts)", fontsize=12, fontweight='bold')
axes[0].set_ylabel("Gercek")
axes[0].set_xlabel("Tahmin")

# Normalized
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues', ax=axes[1],
            xticklabels=['Background', 'Fake'], yticklabels=['Background', 'Fake'])
axes[1].set_title("Confusion Matrix (Normalized)", fontsize=12, fontweight='bold')
axes[1].set_ylabel("Gercek")
axes[1].set_xlabel("Tahmin")

plt.tight_layout()
cm_path = os.path.join(CONFIG['viz_dir'], 'confusion_matrix.png')
plt.savefig(cm_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"Confusion Matrix kaydedildi: {cm_path}")

# Metrikler
tn, fp, fn, tp = cm.ravel()
iou_fake = tp / (tp + fp + fn)
iou_bg = tn / (tn + fp + fn)
mean_iou = (iou_fake + iou_bg) / 2
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print("\n" + "="*50)
print("VALIDATION SET FINAL SONUCLARI")
print("="*50)
print(f"  Fake IoU:       {iou_fake:.4f}")
print(f"  Background IoU: {iou_bg:.4f}")
print(f"  Mean IoU:       {mean_iou:.4f}")
print(f"  Accuracy:       {accuracy:.4f}")
print(f"  Precision:      {precision:.4f}")
print(f"  Recall:         {recall:.4f}")
print(f"  F1-Score:       {f1:.4f}")
print("="*50)

In [ ]:
# =============================================================================
# HUCRE 14: MODEL COMPARISON FUNCTIONS
# =============================================================================
"""
Farklı modellerin performansını karşılaştırma fonksiyonları
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

def load_experiment_metadata(data_dir_pattern):
    """
    Birden fazla experiment metadata dosyasını yükle
    
    Args:
        data_dir_pattern: Experiment klasörlerinin pattern'i (örn: "eğitim_sonuçları/segformer_5080_final/v*")
    
    Returns:
        DataFrame with all experiments
    """
    metadata_files = glob.glob(os.path.join(data_dir_pattern, "**/experiment_metadata.csv"), recursive=True)
    
    if not metadata_files:
        print(f"⚠️ No experiment metadata files found in: {data_dir_pattern}")
        return None
    
    all_experiments = []
    for file in metadata_files:
        try:
            df = pd.read_csv(file)
            all_experiments.append(df)
        except Exception as e:
            print(f"⚠️ Error loading {file}: {e}")
    
    if all_experiments:
        combined_df = pd.concat(all_experiments, ignore_index=True)
        print(f"✅ Loaded {len(combined_df)} experiments from {len(metadata_files)} files")
        return combined_df
    else:
        return None


def compare_models(experiments_df, save_path=None):
    """
    Farklı modellerin performansını karşılaştır
    
    Args:
        experiments_df: Experiment metadata DataFrame
        save_path: Optional path to save comparison plot
    """
    if experiments_df is None or len(experiments_df) == 0:
        print("⚠️ No experiments to compare")
        return None
    
    # Group by model variant
    model_comparison = experiments_df.groupby('model_variant').agg({
        'best_fake_iou': ['mean', 'std', 'max', 'min'],
        'final_mean_iou': ['mean', 'std'],
        'final_accuracy': ['mean', 'std'],
        'experiment_id': 'count'
    }).round(4)
    
    print("\n" + "="*70)
    print("MODEL COMPARISON")
    print("="*70)
    print(model_comparison)
    print("="*70)
    
    if save_path:
        # Visualization
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # 1. Best Fake IoU comparison
        model_variants = experiments_df['model_variant'].unique()
        best_ious = [experiments_df[experiments_df['model_variant'] == m]['best_fake_iou'].values 
                    for m in model_variants]
        
        axes[0, 0].boxplot(best_ious, labels=model_variants)
        axes[0, 0].set_ylabel('Best Fake IoU', fontsize=12, fontweight='bold')
        axes[0, 0].set_title('Best Fake IoU by Model Variant', fontsize=14, fontweight='bold')
        axes[0, 0].grid(True, alpha=0.3, axis='y')
        
        # 2. Mean IoU comparison
        mean_ious = [experiments_df[experiments_df['model_variant'] == m]['final_mean_iou'].values 
                    for m in model_variants]
        axes[0, 1].boxplot(mean_ious, labels=model_variants)
        axes[0, 1].set_ylabel('Mean IoU', fontsize=12, fontweight='bold')
        axes[0, 1].set_title('Mean IoU by Model Variant', fontsize=14, fontweight='bold')
        axes[0, 1].grid(True, alpha=0.3, axis='y')
        
        # 3. Accuracy comparison
        accuracies = [experiments_df[experiments_df['model_variant'] == m]['final_accuracy'].values 
                     for m in model_variants]
        axes[1, 0].boxplot(accuracies, labels=model_variants)
        axes[1, 0].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
        axes[1, 0].set_title('Accuracy by Model Variant', fontsize=14, fontweight='bold')
        axes[1, 0].grid(True, alpha=0.3, axis='y')
        
        # 4. Loss type comparison
        if 'loss_type' in experiments_df.columns:
            loss_types = experiments_df['loss_type'].unique()
            loss_ious = [experiments_df[experiments_df['loss_type'] == lt]['best_fake_iou'].values 
                        for lt in loss_types]
            axes[1, 1].boxplot(loss_ious, labels=loss_types)
            axes[1, 1].set_ylabel('Best Fake IoU', fontsize=12, fontweight='bold')
            axes[1, 1].set_title('Best Fake IoU by Loss Type', fontsize=14, fontweight='bold')
            axes[1, 1].grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"✅ Model comparison plot saved: {save_path}")
    
    return model_comparison


def compare_hyperparameters(experiments_df, save_path=None):
    """
    Farklı hyperparameter kombinasyonlarını karşılaştır
    """
    if experiments_df is None or len(experiments_df) == 0:
        print("⚠️ No experiments to compare")
        return None
    
    # Key hyperparameters
    hyperparams = ['learning_rate', 'batch_size', 'loss_type', 'dice_weight', 'focal_weight']
    available_params = [p for p in hyperparams if p in experiments_df.columns]
    
    if not available_params:
        print("⚠️ No hyperparameter columns found")
        return None
    
    print("\n" + "="*70)
    print("HYPERPARAMETER COMPARISON")
    print("="*70)
    
    for param in available_params:
        if experiments_df[param].notna().any():
            param_comparison = experiments_df.groupby(param).agg({
                'best_fake_iou': ['mean', 'std', 'count'],
                'final_mean_iou': 'mean',
                'final_accuracy': 'mean'
            }).round(4)
            print(f"\n{param.upper()}:")
            print(param_comparison)
    
    print("="*70)
    
    if save_path and len(available_params) > 0:
        # Create correlation heatmap
        numeric_cols = ['best_fake_iou', 'final_mean_iou', 'final_accuracy', 
                       'learning_rate', 'batch_size', 'dice_weight', 'focal_weight']
        numeric_cols = [c for c in numeric_cols if c in experiments_df.columns]
        
        if len(numeric_cols) > 1:
            corr_matrix = experiments_df[numeric_cols].corr()
            
            fig, ax = plt.subplots(figsize=(10, 8))
            sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
                       center=0, square=True, ax=ax)
            ax.set_title('Hyperparameter Correlation Matrix', fontsize=14, fontweight='bold')
            plt.tight_layout()
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.show()
            print(f"✅ Hyperparameter correlation plot saved: {save_path}")
    
    return experiments_df.groupby(available_params[0] if available_params else 'experiment_id').agg({
        'best_fake_iou': 'mean',
        'final_mean_iou': 'mean',
        'final_accuracy': 'mean'
    }).round(4) if available_params else None


print("✅ Model comparison fonksiyonları hazır:")
print("  - load_experiment_metadata()")
print("  - compare_models()")
print("  - compare_hyperparameters()")


In [ ]:
# =============================================================================
# HUCRE 15: DATASET ANALYSIS FUNCTIONS
# =============================================================================
"""
Dataset bazlı performans analizi fonksiyonları
"""

def analyze_dataset_performance(all_labels, all_predictions, all_metadata, save_path=None):
    """
    Farklı dataset'lerden gelen görsellerin performansını analiz et
    
    Args:
        all_labels: True labels
        all_predictions: Predicted labels
        all_metadata: Metadata list (her prediction için)
        save_path: Optional path to save analysis plot
    """
    if not all_metadata or len(all_metadata) == 0:
        print("⚠️ No metadata available for dataset analysis")
        return None
    
    # Extract dataset information
    dataset_data = []
    for i, metadata in enumerate(all_metadata):
        if metadata and 'dataset' in metadata:
            dataset_data.append({
                'dataset': metadata.get('dataset', 'unknown'),
                'true_label': all_labels[i],
                'predicted_label': all_predictions[i],
                'correct': int(all_labels[i] == all_predictions[i]),
                'area_ratio': metadata.get('area_ratio', None),
                'sem_magnitude': metadata.get('sem_magnitude', 'unknown'),
            })
    
    if not dataset_data:
        print("⚠️ No dataset information found in metadata")
        return None
    
    df = pd.DataFrame(dataset_data)
    
    # Dataset bazlı metrikler
    dataset_metrics = df.groupby('dataset').agg({
        'correct': ['count', 'sum', 'mean'],
        'true_label': lambda x: (x == 1).sum()  # Fake count
    }).round(4)
    dataset_metrics.columns = ['total_samples', 'correct_predictions', 'accuracy', 'fake_count']
    dataset_metrics['error_rate'] = 1 - dataset_metrics['accuracy']
    
    print("\n" + "="*70)
    print("DATASET PERFORMANCE ANALYSIS")
    print("="*70)
    print(dataset_metrics.sort_values('accuracy', ascending=False))
    print("="*70)
    
    if save_path:
        # Visualization
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        datasets = dataset_metrics.index.tolist()
        
        # 1. Accuracy by dataset
        axes[0, 0].bar(datasets, dataset_metrics['accuracy'], color='steelblue', alpha=0.7, edgecolor='black')
        axes[0, 0].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
        axes[0, 0].set_title('Accuracy by Dataset', fontsize=14, fontweight='bold')
        axes[0, 0].set_xticklabels(datasets, rotation=45, ha='right')
        axes[0, 0].grid(True, alpha=0.3, axis='y')
        axes[0, 0].set_ylim([0, 1])
        
        # 2. Error rate by dataset
        axes[0, 1].bar(datasets, dataset_metrics['error_rate'], color='coral', alpha=0.7, edgecolor='black')
        axes[0, 1].set_ylabel('Error Rate', fontsize=12, fontweight='bold')
        axes[0, 1].set_title('Error Rate by Dataset', fontsize=14, fontweight='bold')
        axes[0, 1].set_xticklabels(datasets, rotation=45, ha='right')
        axes[0, 1].grid(True, alpha=0.3, axis='y')
        
        # 3. Sample count by dataset
        axes[1, 0].bar(datasets, dataset_metrics['total_samples'], color='lightgreen', alpha=0.7, edgecolor='black')
        axes[1, 0].set_ylabel('Sample Count', fontsize=12, fontweight='bold')
        axes[1, 0].set_title('Sample Count by Dataset', fontsize=14, fontweight='bold')
        axes[1, 0].set_xticklabels(datasets, rotation=45, ha='right')
        axes[1, 0].grid(True, alpha=0.3, axis='y')
        
        # 4. Fake count vs Accuracy scatter
        axes[1, 1].scatter(dataset_metrics['fake_count'], dataset_metrics['accuracy'], 
                          s=dataset_metrics['total_samples']*10, alpha=0.6, edgecolors='black')
        for i, dataset in enumerate(datasets):
            axes[1, 1].annotate(dataset, 
                               (dataset_metrics['fake_count'].iloc[i], 
                                dataset_metrics['accuracy'].iloc[i]),
                               fontsize=8)
        axes[1, 1].set_xlabel('Fake Sample Count', fontsize=12, fontweight='bold')
        axes[1, 1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
        axes[1, 1].set_title('Fake Count vs Accuracy (size=sample count)', fontsize=14, fontweight='bold')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"✅ Dataset analysis plot saved: {save_path}")
    
    # Save to CSV
    csv_path = save_path.replace('.png', '.csv') if save_path else None
    if csv_path:
        dataset_metrics.to_csv(csv_path)
        print(f"✅ Dataset metrics saved: {csv_path}")
    
    return dataset_metrics


def analyze_by_sem_magnitude(all_labels, all_predictions, all_metadata, save_path=None):
    """
    Semantic magnitude'a göre performans analizi
    """
    if not all_metadata or len(all_metadata) == 0:
        print("⚠️ No metadata available")
        return None
    
    magnitude_data = []
    for i, metadata in enumerate(all_metadata):
        if metadata and 'sem_magnitude' in metadata:
            magnitude_data.append({
                'sem_magnitude': metadata.get('sem_magnitude', 'unknown'),
                'true_label': all_labels[i],
                'predicted_label': all_predictions[i],
                'correct': int(all_labels[i] == all_predictions[i]),
            })
    
    if not magnitude_data:
        print("⚠️ No semantic magnitude information found")
        return None
    
    df = pd.DataFrame(magnitude_data)
    magnitude_metrics = df.groupby('sem_magnitude').agg({
        'correct': ['count', 'sum', 'mean']
    }).round(4)
    magnitude_metrics.columns = ['total_samples', 'correct_predictions', 'accuracy']
    
    print("\n" + "="*70)
    print("SEMANTIC MAGNITUDE PERFORMANCE")
    print("="*70)
    print(magnitude_metrics.sort_values('accuracy', ascending=False))
    print("="*70)
    
    if save_path:
        fig, ax = plt.subplots(figsize=(10, 6))
        magnitudes = magnitude_metrics.index.tolist()
        ax.bar(magnitudes, magnitude_metrics['accuracy'], color='mediumpurple', alpha=0.7, edgecolor='black')
        ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
        ax.set_xlabel('Semantic Magnitude', fontsize=12, fontweight='bold')
        ax.set_title('Accuracy by Semantic Magnitude', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='y')
        ax.set_ylim([0, 1])
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"✅ Semantic magnitude analysis saved: {save_path}")
    
    return magnitude_metrics


print("✅ Dataset analysis fonksiyonları hazır:")
print("  - analyze_dataset_performance()")
print("  - analyze_by_sem_magnitude()")


In [ ]:
# =============================================================================
# HUCRE 16: EXPERIMENT METADATA EXPORT (Standalone)
# =============================================================================
"""
Training sonrası experiment metadata export fonksiyonu
MetricsTrackerCallback'te otomatik çağrılmazsa bu fonksiyon kullanılabilir
"""

import datetime

def export_experiment_metadata_standalone(history, best_fake_iou, best_epoch, data_dir):
    """
    Standalone experiment metadata export
    
    Args:
        history: Training history dictionary
        best_fake_iou: Best fake IoU value
        best_epoch: Best epoch number
        data_dir: Directory to save metadata
    """
    # Get final metrics
    final_metrics = {}
    if history and history.get('epoch'):
        final_metrics = {
            'final_train_loss': history['train_loss'][-1] if history.get('train_loss') else None,
            'final_val_loss': history['val_loss'][-1] if history.get('val_loss') else None,
            'final_mean_iou': history['val_mean_iou'][-1] if history.get('val_mean_iou') else None,
            'final_fake_iou': history['val_fake_iou'][-1] if history.get('val_fake_iou') else None,
            'final_bg_iou': history['val_bg_iou'][-1] if history.get('val_bg_iou') else None,
            'final_accuracy': history['val_accuracy'][-1] if history.get('val_accuracy') else None,
            'best_fake_iou': best_fake_iou,
            'best_epoch': best_epoch,
        }
    
    # Create experiment metadata
    experiment_metadata = {
        'experiment_id': CONFIG.get('version', 'unknown'),
        'timestamp': datetime.datetime.now().isoformat(),
        'notebook_name': NOTEBOOK_NAME,
        'dataset_name': CONFIG.get('dataset_name', 'unknown'),
        'model_name': CONFIG.get('model_name', 'unknown'),
        'model_variant': CONFIG.get('model_variant', 'unknown'),
        'img_size': CONFIG.get('img_size', 512),
        'num_classes': CONFIG.get('num_classes', 2),
        'batch_size': CONFIG.get('batch_size', 8),
        'gradient_accumulation': CONFIG.get('gradient_accumulation', 1),
        'effective_batch_size': CONFIG.get('batch_size', 8) * CONFIG.get('gradient_accumulation', 1),
        'num_epochs': CONFIG.get('num_epochs', 10),
        'learning_rate': CONFIG.get('learning_rate', 0.0),
        'weight_decay': CONFIG.get('weight_decay', 0.0),
        'loss_type': CONFIG.get('loss_type', 'unknown'),
        'dice_weight': CONFIG.get('dice_weight', None),
        'focal_weight': CONFIG.get('focal_weight', None),
        'focal_gamma': CONFIG.get('focal_gamma', None),
        'focal_alpha': str(CONFIG.get('focal_alpha', None)),
        'seed': CONFIG.get('seed', 42),
        **final_metrics
    }
    
    # Save to CSV
    df = pd.DataFrame([experiment_metadata])
    csv_path = os.path.join(data_dir, 'experiment_metadata.csv')
    df.to_csv(csv_path, index=False)
    print(f"✅ Experiment metadata saved: {csv_path}")
    
    return df


# Eğitim sonrası otomatik export (eğer MetricsTrackerCallback'te yoksa)
# Bu hücreyi eğitim sonrası çalıştırabilirsiniz:
# export_experiment_metadata_standalone(
#     metrics_callback.history, 
#     metrics_callback.best_fake_iou, 
#     metrics_callback.best_epoch,
#     CONFIG['data_dir']
# )

print("✅ Experiment metadata export fonksiyonu hazır:")
print("  - export_experiment_metadata_standalone()")


In [ ]:
# =============================================================================
# HUCRE 17: MODEL COMPARISON USAGE EXAMPLE
# =============================================================================
"""
Farklı experiment'ları karşılaştırmak için kullanım örneği
"""

# Örnek kullanım (comment out edilmiş - gerektiğinde aktif edin):
"""
# 1. Tüm experiment metadata'larını yükle
base_pattern = "eğitim_sonuçları/segformer_5080_final"
experiments_df = load_experiment_metadata(base_pattern)

if experiments_df is not None:
    # 2. Model karşılaştırması
    model_comp_path = os.path.join(CONFIG['viz_dir'], 'model_comparison.png')
    compare_models(experiments_df, save_path=model_comp_path)
    
    # 3. Hyperparameter karşılaştırması
    hyperparam_path = os.path.join(CONFIG['viz_dir'], 'hyperparameter_comparison.png')
    compare_hyperparameters(experiments_df, save_path=hyperparam_path)
    
    print("\n✅ Model comparison completed!")
else:
    print("⚠️ No experiments found. Run multiple training sessions first.")
"""

print("✅ Model comparison kullanım örneği hazır.")
print("   Birden fazla eğitim sonrası yukarıdaki kodu aktif ederek karşılaştırma yapabilirsiniz.")


In [ ]:
# =============================================================================
# HUCRE 11: MODEL YUKLEME VE INFERENCE ORNEGI
# =============================================================================
# Bu hucre, egitilmis modeli yukleyip tek gorsel uzerinde inference yapar
# Test/production icin kullanilabilir

from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

def load_trained_model(model_path, device='cuda'):
    """Egitilmis modeli ve processor'u yukle"""
    processor = SegformerImageProcessor.from_pretrained(model_path)
    model = SegformerForSemanticSegmentation.from_pretrained(model_path)
    model.to(device)
    model.eval()
    return model, processor

def predict_single_image(image_path, model, processor, device='cuda'):
    """
    Tek bir gorsel icin segmentation tahmini yap
    
    Args:
        image_path: Gorsel yolu
        model: Yuklu SegFormer modeli
        processor: SegFormer processor
        device: cuda veya cpu
    
    Returns:
        pred_mask: Binary tahmin maskesi (0: Background, 1: Fake)
        confidence: Her piksel icin confidence degeri
    """
    # Gorseli yukle
    image = Image.open(image_path).convert("RGB")
    
    # Isle
    inputs = processor(images=image, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Tahmin
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Orijinal boyuta upsample
        upsampled = torch.nn.functional.interpolate(
            logits, size=image.size[::-1],  # (H, W)
            mode='bilinear', align_corners=False
        )
        
        # Softmax ve prediction
        probs = torch.softmax(upsampled, dim=1)
        pred_mask = probs.argmax(dim=1).squeeze().cpu().numpy()
        confidence = probs.max(dim=1)[0].squeeze().cpu().numpy()
    
    return pred_mask, confidence, np.array(image)

def visualize_prediction(image, pred_mask, confidence, save_path=None):
    """
    Tahmin sonucunu gorsellestirir
    """
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # 1. Orijinal
    axes[0].imshow(image)
    axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # 2. Prediction Mask
    axes[1].imshow(pred_mask, cmap='RdYlBu', vmin=0, vmax=1)
    axes[1].set_title('Prediction (Blue=BG, Red=Fake)', fontsize=12, fontweight='bold')
    axes[1].axis('off')
    
    # 3. Confidence Map
    im = axes[2].imshow(confidence, cmap='viridis', vmin=0, vmax=1)
    axes[2].set_title('Confidence Map', fontsize=12, fontweight='bold')
    axes[2].axis('off')
    plt.colorbar(im, ax=axes[2], fraction=0.046)
    
    # 4. Overlay
    overlay = image.copy()
    fake_mask = pred_mask == 1
    overlay[fake_mask, 0] = np.clip(overlay[fake_mask, 0] + 80, 0, 255)
    overlay[fake_mask, 1] = overlay[fake_mask, 1] * 0.7
    overlay[fake_mask, 2] = overlay[fake_mask, 2] * 0.7
    axes[3].imshow(overlay)
    axes[3].set_title('Overlay (Red=Fake)', fontsize=12, fontweight='bold')
    axes[3].axis('off')
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    # Istatistikler
    fake_ratio = fake_mask.sum() / fake_mask.size * 100
    mean_conf = confidence.mean()
    fake_conf = confidence[fake_mask].mean() if fake_mask.sum() > 0 else 0
    
    print(f"\nTahmin Istatistikleri:")
    print(f"  Fake Pixel Orani:    {fake_ratio:.2f}%")
    print(f"  Ortalama Confidence: {mean_conf:.4f}")
    print(f"  Fake Bolge Conf:     {fake_conf:.4f}")


# =============================================================================
# ORNEK KULLANIM
# =============================================================================
# Modeli yukle (egitim tamamlandiysa)
model_path = os.path.join(CONFIG['models_dir'], "final_best_model")

if os.path.exists(model_path):
    print("Egitilmis model yukleniyor...")
    trained_model, trained_processor = load_trained_model(model_path, device)
    print(f"Model yuklendi: {model_path}")
    
    # Test icin rastgele bir gorsel sec
    test_image_path = val_df.iloc[0]['image_path']
    print(f"\nTest gorseli: {test_image_path}")
    
    # Tahmin yap
    pred_mask, confidence, image = predict_single_image(
        test_image_path, trained_model, trained_processor, device
    )
    
    # Gorsellestir
    visualize_prediction(image, pred_mask, confidence)
else:
    print(f"Model bulunamadi: {model_path}")
    print("Once egitimi tamamlayin (HUCRE 8).")